# Setting up the constant
We design the private key for each User to be a 256-bit unsigned integer.

For starters, we design our Merkle-Tree Depth to be 9 level. This will result on having 512 users in total for our system.

In [ ]:
import hashlib
import random
import os
import json
import pandas as pd

In [ ]:
PRIVATE_KEY_BIT_LENGTH = 256
DEPTH = 3
random.seed(13)

# Setting up the Hardhat Node
We want to run the Hardhat Node for our experiment. Using the config file in `hardhat` directory, we choose to have 512 pre-funded accounts.

We will connect to the node and store all of these accounts using the Web3.py library.

In [ ]:
from web3 import Web3

In [ ]:
DEVELOPMENT_HTTP_PROVIDER_URL = 'http://127.0.0.1:8545'

In [ ]:
w3 = Web3(Web3.HTTPProvider(DEVELOPMENT_HTTP_PROVIDER_URL))

In [ ]:
#Make sure we can get the block information using the Web3 Provider
w3.eth.get_block('latest')

In [ ]:
#Fetching the pre-funded accounts (make sure we have 512 accounts for this experiment)
agent_accounts = w3.eth.accounts
len(agent_accounts)

In [ ]:
agent_accounts[0]

# Generating Private Key
Each of the User generate their Private Key by choosing a random integer between \[0,2**256\] and convert it into a bytes array.

In [ ]:
def generate_random_private_key_set():
    leaf_amount = 2**DEPTH
    return [random.getrandbits(PRIVATE_KEY_BIT_LENGTH).to_bytes(int(PRIVATE_KEY_BIT_LENGTH/8),"big") for x in range(leaf_amount)]

In [ ]:
private_key_set = generate_random_private_key_set()
private_key_set

# Deriving Public Key
We derive the Public Key by digesting their respective Private Key using SHA256.

Due to the Private Key Bit Length (256-bit) is shorter than the SHA256 block size (512-bit), we will pad the Private Key with 256-bit of zero value on its right side to create a total of 512-bit input.

We use the following function to calculate the public key:

`public_key = sha256(private_key+zero_padding)`,

where each value has the following length:
- `private_key`: 256-bit (32-byte)
- `zero_padding`: 256-bit (32-byte)
- `public_key`: 256-bit (32-byte)

The Public Key Set represents the registered Public Key in the Authentication System.

In [ ]:
def calculate_public_key(private_key):
    return hashlib.sha256(private_key+int(0).to_bytes(32,"big")).digest()

In [ ]:
public_key_set = [calculate_public_key(x) for x in private_key_set]
public_key_set

We wrap the previous functions into the following function to generate experiment data set.

In [ ]:
def generate_public_private_key_set():
    private_key_set = generate_random_private_key_set()
    public_key_set = [calculate_public_key(pk) for pk in private_key_set]
    return (private_key_set, public_key_set)

In [ ]:
(private_key_set, public_key_set) = generate_public_private_key_set()
(private_key_set, public_key_set)

# Accumulating the Public Key
After calculating the respective Public-Private Key for each User, we accumulate the Public Key Set using an accumulator. In this case, our Accumulator Scheme would be a Merkle-Tree Accumulator.

For simplicity, we implement the most basic Merkle-Tree implementation. We write the algorithm to calculate the Merkle Tree and its proof.

In [ ]:
def calculate_merkle_tree(bytesarray_set):
    merkle_tree = {}
    merkle_tree[0] = bytesarray_set
    for current_level in range(1,DEPTH+1):
        node_amount = 2**(DEPTH-current_level)
        merkle_tree[current_level] = {}
        for node in range(node_amount):
            merkle_tree[current_level][node] = hashlib.sha256(
                merkle_tree[current_level-1][node*2]+
                merkle_tree[current_level-1][(node*2)+1]
            ).digest()
    return merkle_tree

def calculate_direction_selector(index, depth):
    binary_repr = format(index, '0{}b'.format(depth))
    bool_list = [(x=='1') for x in binary_repr]
    bool_list.reverse()
    return bool_list

def calculate_index_path(index, depth):
    binary_repr = format(index, '0{}b'.format(depth))
    index_path = []
    for level in range(depth):
        binary_list = list(binary_repr[:depth-level])
        binary_list[-1] = '1' if binary_list[-1]=='0' else '0'
        neighboor_index_binary_string = ''.join(binary_list)
        neighboor_index = int(neighboor_index_binary_string,2)
        index_path.append(neighboor_index)
    return index_path

def calculate_path(index, depth, merkle_tree):
    index_path = calculate_index_path(index, depth)
    path = []
    for level in range(depth):
        path.append(merkle_tree[level][index_path[level]])
    return path

def verify_membership(public_key, direction_selector, path, root):
    current_digest = public_key
    for x in zip(path, direction_selector):
        (left_digest, right_digest) = (x[0], current_digest) if x[1] else (current_digest, x[0])
        current_digest = hashlib.sha256(left_digest+right_digest).digest()
    return root==current_digest   

On our implementation, we assume that the Public Key Set and its Accumulated Value is publicly available.

In the Merkle-Tree case, the accumulated value is called Merkle-Tree Root, which can be calculated by everyone with the Public Key Set.

In [ ]:
merkle_tree = calculate_merkle_tree(public_key_set)
merkle_tree_root = merkle_tree[DEPTH][0]

# Generating the Membership Proof
The User calculate their Membership Proof using the Accumulator Scheme, in this case the Merkle Proof.

In order to calculate the Merkle Proof value, the User need to know their Public Key index on the Public Key Set.

In [ ]:
user_index_leaf = 7
user = {
    'public_key': public_key_set[user_index_leaf],
    'direction_selector': calculate_direction_selector(user_index_leaf, DEPTH),
    'path': calculate_path(user_index_leaf, DEPTH, merkle_tree),
}
user

The User can check locally that the Membership Proof they calculated is correct. 

In [ ]:
verify_membership(user['public_key'],user['direction_selector'],user['path'], merkle_tree_root)

# Generating the Zero-Knowledge Named Proof

We design our Authentication Proof to be valid only if the User holds the information of the Agent Public Key and the User Private Key. By embedding the Agent Public key as the named of the proof we can prevent replay attack. We call this scheme as zkNamedProof.

To calculate zkNamedProof, we put the Agent Public Key as the name of our Proof. In this case, the Agent's Public Key will be the Ethereum Account Address that consists of 20-bytes (40-characters of hexadecimals) data information.

Due to the SHA256 block size reason in the previous public key calculation process, we also pad the Ethereum Account Address with 12-byte zero value from its left side to create a 32-bytes value (256-bit). Combined with the 256-bit from the private key, we digest both of these value to calculate the digested value.

We use the following function to find the digested value of private key and account address:

`sha256(private_key+(zero_padding+account_address))`,

where each value has the following length:
- `private_key`: 256-bit (32-byte)
- `zero_padding`: 96-bit (12-byte)
- `account_address`: 160-bit (20-byte)

In [ ]:
NAME_BYTE_LENGTH = 20
PAD_BYTE_AMOUNT = 12

In [ ]:
def convert_bytes20_hex_to_bytes(bytes20_hex):
    return int(bytes20_hex,16).to_bytes(NAME_BYTE_LENGTH, "big")

def calculate_digested_value(private_key, account_address):
    padded_account_address = (int(0).to_bytes(PAD_BYTE_AMOUNT,"big"))+ account_address
    return hashlib.sha256(private_key + padded_account_address).digest()

We need to calculate the Knowledge of Digested Value from the Private Key and the Agent's Public Key.

In [ ]:
user_index_leaf = 3
chosen_agent_address_hex = agent_accounts[user_index_leaf]
agent_account_address = convert_bytes20_hex_to_bytes(chosen_agent_address_hex)
user_private_key = private_key_set[user_index_leaf]
user_public_key = public_key_set[user_index_leaf]
digested_value = calculate_digested_value(user_private_key, agent_account_address)
digested_value

This value will be our new private witness that we will use to generate an Authentication Proof

# Compiling User Knowledge to Generate an Authentication Proof
We are now ready to compile all the private witness and the public statement into an Authentication Proof.
We reminds that our Authentication Proof consists of zkMembership Proof and zkNamed Proof.

Given the following information:
- User Public-Private Key Pair, 
- Public Key List, 
- the User Leaf Index of their Public Key in the Public Key List, and 
- the Agent's Account Address

The User will calculate the whole Authentication Proof using the following steps:

In [ ]:
def compile_user_knowledge_npauth(public_key_set, agent_account_address_hex, private_key, public_key, public_key_index):
    merkle_tree = calculate_merkle_tree(public_key_set)
    merkle_tree_root = merkle_tree[DEPTH][0]
    agent_account_address = convert_bytes20_hex_to_bytes(agent_account_address_hex)
    user = {
        'public_statement': {
            'merkle_tree_root': merkle_tree_root,
            'agent_account_address': agent_account_address,
            'agent_account_address_hex': agent_account_address.hex()
        },
        'private_witness': {
            'private_key': private_key,
            'public_key': public_key,
            'direction_selector': calculate_direction_selector(public_key_index, DEPTH),
            'path': calculate_path(public_key_index, DEPTH, merkle_tree),
            'digested_value': calculate_digested_value(private_key,agent_account_address),
        }
    }
    return user

In [ ]:
user_leaf_index = 3
user_private_key = private_key_set[user_leaf_index]
user_public_key = public_key_set[user_leaf_index]
agent_account_address_hex = agent_accounts[user_leaf_index]
user = compile_user_knowledge_npauth(public_key_set, agent_account_address_hex, user_private_key, user_public_key,  user_leaf_index)
user

# Summarizing The User Knowledge Creation Process
We summarize the process in our scheme as follow:
- User generate their private key
- User calculate their public key
- The Administrator registered the public key into the public key set and calculate the Merkle Root
- The User choose an agent to send the Authentication Proof
- The User compile all of their knowledge into an input to ZoKrates

In [ ]:
def pick_random_index_from_merkle_tree():
    return random.randrange(2**DEPTH)

In [ ]:
# Assume the Administrator already have the populated public key set
 
hidden_private_key_list, administrator_public_key_list = generate_public_private_key_set()


random_leaf = pick_random_index_from_merkle_tree()
random_private_key = hidden_private_key_list[random_leaf]

# Assume the User generate a random private key with its respective public key was already registered 
# inside the Administrator's Public Key Set
user_private_key = random_private_key
user_public_key = calculate_public_key(user_private_key)
user_public_key_index = random_leaf
assert(user_public_key==administrator_public_key_list[user_public_key_index])

In [ ]:
# Please change the following address to your account address
user_agent_account_address_hex = agent_accounts[random_leaf]
user_npauth = compile_user_knowledge_npauth(
    administrator_public_key_list, 
    user_agent_account_address_hex, 
    user_private_key, 
    user_public_key,  
    user_public_key_index)
user_npauth

# Zokrates Script

In Zokrates, there are 5 main commands that we need to run:
- `compile`
- `setup`
- `compute-witness`
- `generate-proof`
- `verify`

We want to measure the elapsed-time for these commands to measure its performance.
We use the built-in `%%timeit` command (`%%time` version can be used as alternative for shorter time) in IPython to measure these commands' elapsed-time.

We also wrap all of these command as a function for ease-of-use

Here are the specification of our implementation:
- Backend: Bellman
- Curve: ALT_BN128 (also known as BN254 and BN256)
- Proving Scheme: Groth16

In [ ]:
TEMP_DIRECTORY_PATH = 'temp'
ZOKRATES_DIRECTORY_PATH = 'zokrates'
BUILD_DIRECTORY_PATH = 'builds'
KEY_DIRECTORY_PATH = 'keys'
PROOF_DIRECTORY_PATH = 'proofs'
CONTRACT_DIRECTORY_PATH = 'contracts'

In [ ]:
import subprocess
import shlex

def subprocess_run_wrapper(command, byte_input=None, print_stdout=False):
    process = subprocess.run(shlex.split(command), input=byte_input, stdout=subprocess.PIPE)
    stdout_string = process.stdout.decode('UTF-8')
    if print_stdout:
        print(stdout_string)
    # Make sure the returncode is accepted
    assert(process.returncode == 0)
    return stdout_string

# Compiling ZoKrates Script
We provide the ZoKrates script as the representation of the circuit of our Authentication Proof.
First, we need to run the `zokrates compile` command with the `npauth.zok` as the input.

This command will generate the followings file:
- `abi.json`
- `out`
- `out.r1cs`

We can see the amount of constraint is needed to represent our circuit into the R1CS representation.

In [ ]:
def compile_zokrates_script(scheme_name):
    path = {}
    path['temp_directory'] = TEMP_DIRECTORY_PATH
    path['zokrates_directory'] = ZOKRATES_DIRECTORY_PATH
    path['build_directory'] = BUILD_DIRECTORY_PATH
    path['scheme_name'] = scheme_name

    zokrates_path_string = '{zokrates_directory}/{scheme_name}.zok'.format(**path)
    build_path_string = '{temp_directory}/{build_directory}/{scheme_name}/'.format(**path)
    
    os.system('mkdir -p ' + build_path_string)
    zokrates_compile_command = ('zokrates compile '+
        '-s '+ build_path_string +'abi.json '+
        '-o '+ build_path_string +'out '+
        '-r '+ build_path_string +'out.r1cs '+
        '-i '+ zokrates_path_string)
    stdout_string = subprocess_run_wrapper(zokrates_compile_command)
    circuit_size = int(stdout_string.split(' ')[-1].replace('\n',''))
    return circuit_size

In [ ]:
circuit_size = compile_zokrates_script('npauth')
circuit_size

# Generating keys using Trusted Setup
We want to generate the proving key and the verification key using the Trusted Setup. 

We run the `zokrates setup` command with the previous `out` file from the `compile` command as an input.
We will use the `bellman` backend for this implementation due to the other backend `ark` is buggy in our experience.

In [ ]:
def generate_proving_verification_key_with_trusted_setup(scheme_name):
    path = {}
    path['temp_directory'] = TEMP_DIRECTORY_PATH
    path['zokrates_directory'] = ZOKRATES_DIRECTORY_PATH
    path['build_directory'] = BUILD_DIRECTORY_PATH
    path['scheme_name'] = scheme_name
    path['key_directory'] = KEY_DIRECTORY_PATH

    build_path_string = '{temp_directory}/{build_directory}/{scheme_name}/'.format(**path)
    key_path_string = '{temp_directory}/{key_directory}/{scheme_name}/'.format(**path)
    os.system('mkdir -p ' + key_path_string)
    zokrates_setup_command = ('zokrates setup '+
        '-i '+ build_path_string +'out '+
        '--backend bellman ' +
        '--proving-key-path '+ key_path_string +'proving.key '+
        '--verification-key-path '+ key_path_string +'verification.key')
    stdout_string = subprocess_run_wrapper(zokrates_setup_command)

def check_keys_size(scheme_name):
    path = {}
    path['temp_directory'] = TEMP_DIRECTORY_PATH
    path['key_directory'] = KEY_DIRECTORY_PATH
    path['scheme_name'] = scheme_name
    
    key_path_string = '{temp_directory}/{key_directory}/{scheme_name}/'.format(**path)
    
    pk_size = os.path.getsize(key_path_string + 'proving.key')
    vk_size = os.path.getsize(key_path_string + 'verification.key')
    print('Proving Key Size in Bytes: {:,}'.format(pk_size))
    print('Verification Key Size in Bytes: {:,}'.format(vk_size))
    return (pk_size, vk_size)
    

In [ ]:
generate_proving_verification_key_with_trusted_setup('npauth')

In [ ]:
check_keys_size('npauth')

# Generating the proof using ZoKrates
Using the previous user information, we will create a proof using ZoKrates.

In [ ]:
def convert_bytes_array_to_u32_array(bytes_array):
    four_bytes_array = [bytes_array[x:x+4] for x in range(0,len(bytes_array),4)]
    uint32_array = [int().from_bytes(x,"big") for x in four_bytes_array]
    return uint32_array

def convert_bytes_array_to_u32_string_array(bytes_array):
    return [str(u32_array) for u32_array in convert_bytes_array_to_u32_array(bytes_array)]

In [ ]:
def generate_proof_from_user_knowledge(scheme_name, scheme_format_function, scheme_proof_subdirectory_path, user_knowledge_dictionary):
    path = {}
    path['temp_directory'] = TEMP_DIRECTORY_PATH
    path['zokrates_directory'] = ZOKRATES_DIRECTORY_PATH
    path['build_directory'] = BUILD_DIRECTORY_PATH
    path['key_directory'] = KEY_DIRECTORY_PATH
    path['scheme_name'] = scheme_name
    path['proof_directory'] = PROOF_DIRECTORY_PATH
    path['public_key_hex'] = user_knowledge_dictionary['private_witness']['public_key'].hex()
    path['scheme_proof_subdirectory'] = scheme_proof_subdirectory_path.format(**user_knowledge_dictionary)
    
    build_path_string = '{temp_directory}/{build_directory}/{scheme_name}/'.format(**path)
    key_path_string = '{temp_directory}/{key_directory}/{scheme_name}/'.format(**path)
    proof_path_string = '{temp_directory}/{proof_directory}/{scheme_name}/{public_key_hex}{scheme_proof_subdirectory}/'.format(**path)

    # Calculating input array using the scheme's format function
    zokrates_input_array = scheme_format_function(user_knowledge_dictionary)
    zokrates_json_string = json.dumps(zokrates_input_array)

    os.system('mkdir -p ' + proof_path_string)

    # Generating witness from input.json    
    zokrates_compute_witness_command = ('zokrates compute-witness '+
    '-s '+ build_path_string + 'abi.json '+
    '-i '+ build_path_string +'out '+
    '--circom-witness '+ build_path_string +'out.wtns '+
    '-o ' + proof_path_string + 'witness ' +
    '--verbose --abi '+
    '--stdin')
    
    stdout_string = subprocess_run_wrapper(zokrates_compute_witness_command, zokrates_json_string.encode())
    assert(stdout_string.split('\n')[3]=='true')
    
    
    # Generating proof from witness
    # Check if proof_output.json file exists and not overwriting existing files
    zokrates_generate_proof_command = ('zokrates generate-proof '+
    '--backend bellman ' +
    '-i '+ build_path_string +'out '+
    '-w '+ proof_path_string +'witness '+
    '-p '+ key_path_string + 'proving.key '+
    '-j '+ proof_path_string +'proof_output.json')
    subprocess_run_wrapper(zokrates_generate_proof_command)
    

In [ ]:
def format_npauth_user_knowledge_into_zokrates_input_array(user):
    zokrates_input = [
        convert_bytes_array_to_u32_string_array(user['public_statement']['merkle_tree_root']),
        convert_bytes_array_to_u32_string_array(user['public_statement']['agent_account_address']),
        convert_bytes_array_to_u32_string_array(user['private_witness']['private_key']),
        convert_bytes_array_to_u32_string_array(user['private_witness']['public_key']),
        user['private_witness']['direction_selector'],
        [convert_bytes_array_to_u32_string_array(bytes_array) for bytes_array in user['private_witness']['path']],
        convert_bytes_array_to_u32_string_array(user['private_witness']['digested_value']),
    ]
    return zokrates_input

In [ ]:
generate_proof_from_user_knowledge('npauth', format_npauth_user_knowledge_into_zokrates_input_array, '/{public_statement[agent_account_address_hex]}', user_npauth)

In [ ]:
def verify_generated_proof_with_verification_key(scheme_name, scheme_proof_subdirectory_path, user_knowledge_dictionary):
    path = {}
    path['temp_directory'] = TEMP_DIRECTORY_PATH
    path['zokrates_directory'] = ZOKRATES_DIRECTORY_PATH
    path['key_directory'] = KEY_DIRECTORY_PATH
    path['scheme_name'] = scheme_name
    path['proof_directory'] = PROOF_DIRECTORY_PATH
    path['public_key_hex'] = user_knowledge_dictionary['private_witness']['public_key'].hex()
    path['scheme_proof_subdirectory'] = scheme_proof_subdirectory_path.format(**user_knowledge_dictionary)

    key_path_string = '{temp_directory}/{key_directory}/{scheme_name}/'.format(**path)
    proof_path_string = '{temp_directory}/{proof_directory}/{scheme_name}/{public_key_hex}{scheme_proof_subdirectory}/'.format(**path)
    
    zokrates_verify_command = ('zokrates verify '+
    '-v '+ key_path_string +'verification.key '+
    '-j '+ proof_path_string +'proof_output.json')
    return subprocess_run_wrapper(zokrates_verify_command)

In [ ]:
verify_generated_proof_with_verification_key('npauth', '/{public_statement[agent_account_address_hex]}', user_npauth)

# Generating Verification Contract
ZoKrates provides a way to generate a Smart Contract that can verify our proof in one command.

We run the command `zokrates export-verifier` to generate the Solidity file called `verifier.sol`. We contain this Solidity file on `contracts` directory.

In [ ]:
def generate_verification_contract_from_verification_key(scheme_name):
    path = {}
    path['temp_directory'] = TEMP_DIRECTORY_PATH
    path['scheme_name'] = scheme_name
    path['key_directory'] = KEY_DIRECTORY_PATH
    path['contract_path'] = CONTRACT_DIRECTORY_PATH

    contract_path_string = '{temp_directory}/{contract_path}/'.format(**path)
    key_path_string = '{temp_directory}/{key_directory}/{scheme_name}/'.format(**path)
    contract_name = '{scheme_name}_verifier.sol'.format(**path)
    
    #Check if verifier.sol file exists and not overwriting existing files
    os.system('mkdir -p ' + contract_path_string)
    zokrates_export_verifier_command = ('zokrates export-verifier '
    '-i '+ key_path_string +'verification.key '
    '-o '+ contract_path_string + contract_name)
    return subprocess_run_wrapper(zokrates_export_verifier_command)

In [ ]:
generate_verification_contract_from_verification_key('npauth')

# Deploying the Blockchain-Based Authentication Scheme
Using the previous contract, we can extend it to create an authentication scheme. We provide our source code in the `contracts` directory called `npauth.sol`.

We will deploy the previous smart contract using Web3.py.

Before deploying the smart contract, we need to compile the smart contract using the following commands.

In [ ]:
from solcx import install_solc, compile_files, set_solc_version
install_solc(version='0.8.24')
set_solc_version('0.8.24')
def compile_and_deploy_authentication_contract(scheme_name, class_name, web3_http_provider, return_receipt=False):
    path = {}
    path['scheme_name'] = scheme_name
    path['class_name'] = class_name
    path['contract_directory'] = CONTRACT_DIRECTORY_PATH
    compiled = compile_files([os.getcwd()+"/{contract_directory}/{scheme_name}.sol".format(**path)], output_values=["abi", "bin"], allow_paths = os.getcwd()+"/temp/contracts")
    output = compiled['{contract_directory}/{scheme_name}.sol:{class_name}'.format(**path)]
    compiled_web3_contract = web3_http_provider.eth.contract(abi=output['abi'], bytecode=output['bin'])
    # We use the default account to send the transaction (e.g., the first account in the HD wallet)
    tx_hash = compiled_web3_contract.constructor().transact()
    tx_receipt = web3_http_provider.eth.wait_for_transaction_receipt(tx_hash)
    deployed_web3_contract = web3_http_provider.eth.contract(address=tx_receipt.contractAddress, abi=output['abi'])
    if return_receipt:
        return (deployed_web3_contract, tx_receipt)
    return deployed_web3_contract

In [ ]:
npauth_contract = compile_and_deploy_authentication_contract('npauth', 'NPAuth', w3)

# Interacting with the Authentication Contract

After deploying the previous authentication contract, we need to interact with the smart contract.

We give some mandatory functions to interact with our authentication contract.

In [ ]:
def update_current_merkle_root_of_the_auth_contract(web3_contract, merkle_root_bytes, web3_http_provider):
    # We use the default account to send the transaction (e.g., the first account in the HD wallet)
    tx_hash = web3_contract.functions.update_current_merkle_root(merkle_root_bytes).transact()
    tx_receipt = web3_http_provider.eth.wait_for_transaction_receipt(tx_hash)
    return tx_receipt

def connect_to_contract(contract_address, contract_abi, web3_http_provider):
    return web3_http_provider.eth.contract(address=contract_address, abi=contract_abi)

Our previous function help us to update the merkle root in the authentication contract. The previous function should be done only by administrator in the real-world.

In [ ]:
update_current_merkle_root_of_the_auth_contract(npauth_contract, user_npauth['public_statement']['merkle_tree_root'], w3)

In [ ]:
#Make sure these two values is identical after updating the merkle root
npauth_contract.functions.current_merkle_root().call().hex()==user_npauth['public_statement']['merkle_tree_root'].hex()

After updating the merkle root, now we will try to send our authentication request to the authentication contract. We do this by first parsing the `proof_output.json` file from the ZoKrates output in the previous step. We give the functions below to do this thing.

In [ ]:
def parse_proof_output_json_into_contract_input(scheme_name, scheme_format_function, scheme_proof_subdirectory_path, user_knowledge_dictionary):
    path = {}
    path['temp_directory'] = TEMP_DIRECTORY_PATH
    path['zokrates_directory'] = ZOKRATES_DIRECTORY_PATH
    path['build_directory'] = BUILD_DIRECTORY_PATH
    path['key_directory'] = KEY_DIRECTORY_PATH
    path['scheme_name'] = scheme_name
    path['proof_directory'] = PROOF_DIRECTORY_PATH
    path['public_key_hex'] = user_knowledge_dictionary['private_witness']['public_key'].hex()
    path['scheme_proof_subdirectory'] = scheme_proof_subdirectory_path.format(**user_knowledge_dictionary)

    proof_path_string = '{temp_directory}/{proof_directory}/{scheme_name}/{public_key_hex}{scheme_proof_subdirectory}/'.format(**path)
    
    with open(proof_path_string + 'proof_output.json') as f:
        parsed_json = json.load(f)
    g1_point_a = (int(parsed_json['proof']['a'][0],16), int(parsed_json['proof']['a'][1],16))
    g2_point_b = ([int(x,16) for x in parsed_json['proof']['b'][0]], [int(x,16) for x in parsed_json['proof']['b'][1]])
    g1_point_c = (int(parsed_json['proof']['c'][0],16), int(parsed_json['proof']['c'][1],16))
    proof = (g1_point_a, g2_point_b, g1_point_c)
    hex_lists = parsed_json['inputs']
    inputs = [int(x, 16) for x in hex_lists]
    return proof, inputs

def send_authentication_request_using_designated_agent(web3_http_provider, deployed_contract, proof, inputs, designated_agent_address):
    # In this case, we can't use default account like the previous example
    tx_hash = deployed_contract.functions.authentication_with_broadcast(proof, inputs).transact({"from": designated_agent_address})
    tx_receipt = web3_http_provider.eth.wait_for_transaction_receipt(tx_hash)
    return tx_receipt

Running the command below will send the transaction request to the authentication contract. If the authentication process is successful, the amount of gas used is around 380k gas.

In [ ]:
np_proof, np_inputs = parse_proof_output_json_into_contract_input('npauth', format_npauth_user_knowledge_into_zokrates_input_array, '/{public_statement[agent_account_address_hex]}', user_npauth)
send_authentication_request_using_designated_agent(w3, npauth_contract, np_proof, np_inputs, user_agent_account_address_hex)

Using different agent to submit this authentication request will result in fail due to the safety mechanism inside the smart contract. Below are the example of such things. The gas used is only 58k compared to 380k.

In [ ]:
send_authentication_request_using_designated_agent(w3, npauth_contract, np_proof, np_inputs, agent_accounts[0])

# Other Scheme Implementation
We implement the different scheme as well in our experiment.

## HashAuth Implementation (REMOVED)

This scheme is removed in the manuscript due to its malleability vulnerability.

Due to the different logic in the circuit, we need to remove the agent information from the user to generate a correct zokrates input for other scheme.

We will run the following command to generate the HashAuth scheme's zokrates input.

In [ ]:
def compile_user_knowledge_hashauth(public_key_set, agent_account_address_hex, private_key, public_key, public_key_index):
    merkle_tree = calculate_merkle_tree(public_key_set)
    merkle_tree_root = merkle_tree[DEPTH][0]
    user = {
        'public_statement': {
            'merkle_tree_root': merkle_tree_root,
            'random': random.getrandbits(160).to_bytes(20, "big").hex()
        },
        'private_witness': {
            'private_key': private_key,
            'public_key': public_key,
            'direction_selector': calculate_direction_selector(public_key_index, DEPTH),
            'path': calculate_path(public_key_index, DEPTH, merkle_tree),
        }
    }
    return user

In [ ]:
def format_hashauth_user_knowledge_into_zokrates_input_array(user):
    zokrates_input = [
        convert_bytes_array_to_u32_string_array(user['public_statement']['merkle_tree_root']),
        convert_bytes_array_to_u32_string_array(user['private_witness']['private_key']),
        convert_bytes_array_to_u32_string_array(user['private_witness']['public_key']),
        user['private_witness']['direction_selector'],
        [convert_bytes_array_to_u32_string_array(bytes_array) for bytes_array in user['private_witness']['path']],
    ]
    return zokrates_input

### Setup Phase
We need to run the following command once to do the following things:
- Script Compilation
- Trusted Setup Process
- Verification Contract Generation

In [ ]:
compile_zokrates_script('hashauth')

In [ ]:
generate_proving_verification_key_with_trusted_setup('hashauth')

In [ ]:
generate_verification_contract_from_verification_key('hashauth')

In [ ]:
check_keys_size('hashauth')

### Proof Generation Phase
The following process need to be done multiple times. The witness need to be calculated every time the Administrator update the merkle root, while the proof generation process is done on every access request.

In [ ]:
user_hashauth = compile_user_knowledge_hashauth(
    administrator_public_key_list, 
    user_agent_account_address_hex, 
    user_private_key, 
    user_public_key,  
    user_public_key_index)
user_hashauth

In [ ]:
generate_proof_from_user_knowledge('hashauth', format_hashauth_user_knowledge_into_zokrates_input_array, '/{public_statement[random]}', user_hashauth)

In [ ]:
verify_generated_proof_with_verification_key('hashauth', '/{public_statement[random]}', user_hashauth)

Use the previous codes to deploy, update, and send the authentication request. 

In [ ]:
hashauth_contract = compile_and_deploy_authentication_contract('hashauth', 'HashAuth', w3)
update_current_merkle_root_of_the_auth_contract(hashauth_contract, user['public_statement']['merkle_tree_root'], w3)
hash_proof, hash_inputs = parse_proof_output_json_into_contract_input('hashauth', format_hashauth_user_knowledge_into_zokrates_input_array, '/{public_statement[random]}', user_hashauth)
send_authentication_request_using_designated_agent(w3, hashauth_contract, hash_proof, hash_inputs, user_agent_account_address_hex)

## Nullifier NonceAuth Implementation
We implement our third implementation from Anonymous Parking Authentication, which we called Nullifier NonceAuth, due to its core idea of using rotating nonce as a nullifier value to the smart contract.

On the circuit logic, this implementation is identical to the NPAuth. The two scheme use the following 2 public information:
- Current Merkle Root
- Nonce or Agent Address

For the experiment purpose, we decide the Nonce Length in the circuit to be identical to the Agent Address Length (both 20-bytes length).

The main difference of these schemes lies on the Smart Contract when evaluating the nonce or the agent address. We explained the logic of the rotating nonce in the main paper.

We wrote a zokrates script called `nullifier_nonceauth.zok` by renaming the `agent_account_address` into `rotating_nonce` in the script. We choose the same lenght of the rotating nonce to make a better comparison between each scheme.

### Setup Phase
We run the similar setup phase as the previous scheme.

In [ ]:
compile_zokrates_script('nullifier_nonceauth')

In [ ]:
generate_proving_verification_key_with_trusted_setup('nullifier_nonceauth')

In [ ]:
generate_verification_contract_from_verification_key('nullifier_nonceauth')

In [ ]:
check_keys_size('nullifier_nonceauth')

### Proof Generation Phase
Unlike the previous scheme, we need to always update the nonce to the current_nonce on blockchain.

In [ ]:
def compile_user_knowledge_nullifier_nonceauth(public_key_set, private_key, public_key, public_key_index, nonce_hex):
    merkle_tree = calculate_merkle_tree(public_key_set)
    merkle_tree_root = merkle_tree[DEPTH][0]
    nonce_bytes_array = convert_bytes20_hex_to_bytes(nonce_hex)
    user = {
        'public_statement': {
            'merkle_tree_root': merkle_tree_root,
            'nonce': nonce_bytes_array,
            'nonce_hex': nonce_bytes_array.hex()
        },
        'private_witness': {
            'private_key': private_key,
            'public_key': public_key,
            'direction_selector': calculate_direction_selector(public_key_index, DEPTH),
            'path': calculate_path(public_key_index, DEPTH, merkle_tree),
            'digested_value': calculate_digested_value(private_key,nonce_bytes_array),
        }
    }
    return user

def format_nullifier_nonceauth_user_knowledge_into_zokrates_input_array(user):
    zokrates_input = [
        convert_bytes_array_to_u32_string_array(user['public_statement']['merkle_tree_root']),
        convert_bytes_array_to_u32_string_array(user['public_statement']['nonce']),
        convert_bytes_array_to_u32_string_array(user['private_witness']['private_key']),
        convert_bytes_array_to_u32_string_array(user['private_witness']['public_key']),
        user['private_witness']['direction_selector'],
        [convert_bytes_array_to_u32_string_array(bytes_array) for bytes_array in user['private_witness']['path']],
        convert_bytes_array_to_u32_string_array(user['private_witness']['digested_value']),
    ]
    return zokrates_input

In [ ]:
# Generate random nonce for the sake of initialization
random_integer = random.getrandbits(160)
random_hex_from_random_integer = random_integer.to_bytes(20, "big").hex()
nonce_hex = random_hex_from_random_integer
user_nonce = compile_user_knowledge_nullifier_nonceauth(
    administrator_public_key_list,  
    user_private_key, 
    user_public_key,  
    user_public_key_index,
    nonce_hex)
user_nonce

In [ ]:
generate_proof_from_user_knowledge('nullifier_nonceauth', format_nullifier_nonceauth_user_knowledge_into_zokrates_input_array, '/{public_statement[nonce_hex]}', user_nonce)

In [ ]:
verify_generated_proof_with_verification_key('nullifier_nonceauth', '/{public_statement[nonce_hex]}', user_nonce)

We need to add one more function to update the current nonce on this authentication contract, which is nearly identical to the `update_current_merkle_root_of_the_auth_contract()`.

In [ ]:
def update_current_nonce_of_the_auth_contract(web3_contract, nonce_bytes, web3_http_provider):
    # We use the default account to send the transaction (e.g., the first account in the HD wallet)
    tx_hash = web3_contract.functions.update_current_nonce(nonce_bytes).transact()
    tx_receipt = web3_http_provider.eth.wait_for_transaction_receipt(tx_hash)
    return tx_receipt

In [ ]:
nullifier_nonceauth_contract = compile_and_deploy_authentication_contract('nullifier_nonceauth', 'NullifierNonceAuth', w3)
update_current_merkle_root_of_the_auth_contract(nullifier_nonceauth_contract, user_nonce['public_statement']['merkle_tree_root'], w3)
update_current_nonce_of_the_auth_contract(nullifier_nonceauth_contract, user_nonce['public_statement']['nonce'], w3)
nullifier_nonceauth_proof, nullifier_nonceauth_inputs = parse_proof_output_json_into_contract_input('nullifier_nonceauth', format_nullifier_nonceauth_user_knowledge_into_zokrates_input_array, '/{public_statement[nonce_hex]}', user_nonce)
send_authentication_request_using_designated_agent(w3, nullifier_nonceauth_contract, nullifier_nonceauth_proof, nullifier_nonceauth_inputs, user_agent_account_address_hex)

## Nullifier RandomAuth
We implement the replay attack prevention from the work of Huang et. al., the AASmartCity, which use the same nullifier technique as the AnonParking implementation. We remind that this is not the whole implementation but only the replay attack part, combined with our membership authentication style.

In [ ]:
compile_zokrates_script('nullifier_randomauth')

In [ ]:
generate_proving_verification_key_with_trusted_setup('nullifier_randomauth')

In [ ]:
generate_verification_contract_from_verification_key('nullifier_randomauth')

In [ ]:
check_keys_size('nullifier_randomauth')

In [ ]:
def compile_user_knowledge_nullifier_randomauth(public_key_set, private_key, public_key, public_key_index, aid_hex):
    merkle_tree = calculate_merkle_tree(public_key_set)
    merkle_tree_root = merkle_tree[DEPTH][0]
    aid_bytes_array = convert_bytes20_hex_to_bytes(aid_hex)
    user = {
        'public_statement': {
            'merkle_tree_root': merkle_tree_root,
            'aid': aid_bytes_array,
            'aid_hex': aid_bytes_array.hex()
        },
        'private_witness': {
            'private_key': private_key,
            'public_key': public_key,
            'direction_selector': calculate_direction_selector(public_key_index, DEPTH),
            'path': calculate_path(public_key_index, DEPTH, merkle_tree),
            'digested_value': calculate_digested_value(private_key,aid_bytes_array),
        }
    }
    return user

def format_nullifier_randomauth_user_knowledge_into_zokrates_input_array(user):
    zokrates_input = [
        convert_bytes_array_to_u32_string_array(user['public_statement']['merkle_tree_root']),
        convert_bytes_array_to_u32_string_array(user['public_statement']['aid']),
        convert_bytes_array_to_u32_string_array(user['private_witness']['private_key']),
        convert_bytes_array_to_u32_string_array(user['private_witness']['public_key']),
        user['private_witness']['direction_selector'],
        [convert_bytes_array_to_u32_string_array(bytes_array) for bytes_array in user['private_witness']['path']],
        convert_bytes_array_to_u32_string_array(user['private_witness']['digested_value']),
    ]
    return zokrates_input

In [ ]:
# Generate random nullifier that we use as authentication ID
authentication_id = random.getrandbits(160)
aid_hex = authentication_id.to_bytes(20, "big").hex()
assert(authentication_id==int(aid_hex,16))
user_random = compile_user_knowledge_nullifier_randomauth(
    administrator_public_key_list, 
    user_private_key, 
    user_public_key,  
    user_public_key_index, 
    aid_hex)
user_random

In [ ]:
generate_proof_from_user_knowledge('nullifier_randomauth', format_nullifier_randomauth_user_knowledge_into_zokrates_input_array, '/{public_statement[aid_hex]}', user_random)

In [ ]:
verify_generated_proof_with_verification_key('nullifier_randomauth', '/{public_statement[aid_hex]}', user_random)

In [ ]:
nullifier_randomauth_contract = compile_and_deploy_authentication_contract('nullifier_randomauth', 'NullifierRandomAuth', w3)
update_current_merkle_root_of_the_auth_contract(nullifier_randomauth_contract, user_random['public_statement']['merkle_tree_root'], w3)
nullifier_randomauth_proof, nullifier_randomauth_inputs = parse_proof_output_json_into_contract_input('nullifier_randomauth', format_nullifier_randomauth_user_knowledge_into_zokrates_input_array, '/{public_statement[aid_hex]}', user_random)
send_authentication_request_using_designated_agent(w3, nullifier_randomauth_contract, nullifier_randomauth_proof, nullifier_randomauth_inputs, user_agent_account_address_hex)

## Nullifier PseudoAuth
We implement the replay attack prevention from the work of Luong et. al., the PseudoAuth, which is also a nullifier technique. However, unlike the other authentication scheme, each User may only submit their zero-knowledge proof once to authenticate their blockchain account. This blockchain account is the Pseudonym, an authenticated blockchain account, which can be used by the User to access all kind functionality in the smart contract.

There are some difference however on how the nullifier is calculated in this work. We implement the work using the following calculation.

In [ ]:
def compile_user_knowledge_nullifier_pseudoauth(public_key_set, private_key, public_key, public_key_index, nullifier):
    merkle_tree = calculate_merkle_tree(public_key_set)
    merkle_tree_root = merkle_tree[DEPTH][0]
    user = {
        'public_statement': {
            'merkle_tree_root': merkle_tree_root,
            'nullifier': nullifier,
            'nullifier_hex': nullifier.hex(),
        },
        'private_witness': {
            'private_key': private_key,
            'public_key': public_key,
            'direction_selector': calculate_direction_selector(public_key_index, DEPTH),
            'path': calculate_path(public_key_index, DEPTH, merkle_tree),
        }
    }
    return user

def format_nullifier_pseudoauth_user_knowledge_into_zokrates_input_array(user):
    zokrates_input = [
        convert_bytes_array_to_u32_string_array(user['public_statement']['merkle_tree_root']),
        convert_bytes_array_to_u32_string_array(user['public_statement']['nullifier']),
        convert_bytes_array_to_u32_string_array(user['private_witness']['private_key']),
        convert_bytes_array_to_u32_string_array(user['private_witness']['public_key']),
        user['private_witness']['direction_selector'],
        [convert_bytes_array_to_u32_string_array(bytes_array) for bytes_array in user['private_witness']['path']],
    ]
    return zokrates_input

In [ ]:
nullifier = hashlib.sha256(user_private_key+int(1).to_bytes(int(256/8),"big")).digest()
user_pseudo = compile_user_knowledge_nullifier_pseudoauth(
    administrator_public_key_list, 
    user_private_key, 
    user_public_key,  
    user_public_key_index,
    nullifier)
user_pseudo

In [ ]:
compile_zokrates_script('nullifier_pseudoauth')

In [ ]:
generate_proving_verification_key_with_trusted_setup('nullifier_pseudoauth')

In [ ]:
generate_verification_contract_from_verification_key('nullifier_pseudoauth')

In [ ]:
check_keys_size('nullifier_pseudoauth')

In [ ]:
generate_proof_from_user_knowledge('nullifier_pseudoauth',format_nullifier_pseudoauth_user_knowledge_into_zokrates_input_array,'/{public_statement[nullifier_hex]}',user_pseudo)

In [ ]:
verify_generated_proof_with_verification_key('nullifier_pseudoauth','/{public_statement[nullifier_hex]}',user_pseudo)

In [ ]:
nullifier_pseudoauth_contract = compile_and_deploy_authentication_contract('nullifier_pseudoauth', 'NullifierPseudoAuth', w3)
update_current_merkle_root_of_the_auth_contract(nullifier_pseudoauth_contract, user['public_statement']['merkle_tree_root'], w3)
nullifier_pseudoauth_proof, nullifier_pseudoauth_inputs = parse_proof_output_json_into_contract_input('nullifier_pseudoauth',format_nullifier_pseudoauth_user_knowledge_into_zokrates_input_array,'/{public_statement[nullifier_hex]}',user_pseudo)
send_authentication_request_using_designated_agent(w3, nullifier_pseudoauth_contract, nullifier_pseudoauth_proof, nullifier_pseudoauth_inputs, user_agent_account_address_hex)

## Nullifier TokenAuth
We implement the work from the Token-based Authentication Scheme. 

In order to authenticate, the User need to submit the following value into the smart contract:
- Merkle Root `rt`
- Serial Number `sn` (served as a flag to check if the proof is still valid or not)
- Capability Token Commitment `cm`
- Word Commitment `wm` (served as a nullifier in this scheme)
- zero-knowledge proof of Token Ownership

To prove they have appropriate token, the User need to hold the following private information:
- zeroToken `t` that comprise of:
    - User Public key `a_pk`, which we renamed to `public_key`
    - Capability Token `cm` (used as the public statement)
    - `rho` as the private value of `sn`
    - and `r` as private value of `tm`
    - zeroToken Commitment `tm` (the leaf index in the Merkle Tree, proven using set-membership proof)
- User Private key `a_sk`, which we renamed to `private_key`
- secret word `w` (served as the private value of `wm`)

For simplicity, we assume that all of the previous values have 256-bit length.

We assume that each user is given a token by the administrator.

There will be multiple sets that we need to track in our experiment, which have the length of the total amount of token exists:
- `cm_set`: Consists of all Capability token from all users. 
- `rho_set`: Consists of all random value of `sn` from all users. Unlike the other set, the length of this set would grow if there is a delegation process occur.
- `r_set`: Consists of all random value of `tm` from all users. 
- `tm_set`: Consists of all zeroToken commitment from all users. 


We begin our implementation by creating a pair of User private key `a_sk` and its public key `a_pk`, which is also identical to our work.

In [ ]:
# Get a random private key and public key
random_user_leaf = pick_random_index_from_merkle_tree()
a_sk = hidden_private_key_list[random_user_leaf]
a_pk = administrator_public_key_list[random_user_leaf]
(a_sk,a_pk)

This work also use the nullifier technique to prevent replay attack, which in their work they called as Word Commitment. The word commitment value is always random on each authentication session, thus an identical word commitment value implies replayed proof.

We calculate Word Commitment `wm` with the following calculation:
- `wm = SHA256(k')`
- where `k_prime` is:
    - `k_prime = SHA256(a_sk, w)`
    - where `a_sk` and `w` is the User private key and random nullifier value respectively.

In [ ]:
# Get a random 256-bit value for the random nullifier w
# This value is new on every authentication requests
w = random.getrandbits(256).to_bytes(int(256/8),"big")
k_prime = hashlib.sha256(a_sk+w).digest()
wm = hashlib.sha256(k_prime+int(0).to_bytes(32,"big")).digest()
wm

Due to this work delegation technique, we also need to make sure that the token is not delegated yet, which can be checked using the Serial Number `sn`.

We calculate the value `sn` using the following formula: `sn = SHA256(rho, a_sk)`, where rho is a random number attached to the token.

In [ ]:
# Get a random 256-bit value for the rho
# This value is not changed if the token is not delegated
rho_set = [random.getrandbits(256).to_bytes(int(256/8),"big") for x in range(2**DEPTH)]
rho = rho_set[random_user_leaf]
sn = hashlib.sha256(rho+a_sk).digest()
sn

In the original work, the CapToken Commitment value is calculated as follow:
- cm = SHA256(capToken||rn), where:
    - capToken is a tuple containing (object, resource, action)
    - rn is a random value

The administrator is the only entity that can issued a token. 

This capability token will be accumulated using a merkle tree after converted into a zeroToken commitment `tm`.

The amount of token and the amount of user in the system may be different from each other.

We create the dummy version of Capability Token Commitment `cm` using the following code. 

In [ ]:
# We create a dummy CapToken Commitment cm using 256-bit data instead of using the original calculation
# This value is generated by the adminsistrator
cm_set = [random.getrandbits(256).to_bytes(int(256/8),"big") for x in range(2**DEPTH)]
cm = cm_set[random_user_leaf]

In the literature, the zeroToken Commitment `tm` is calculated using the following calculation:
- `tm = SHA256(cm,k)`
    - where k is
        - `k = SHA256(a_pk, rho, r)`
        
However, due to how SHA256 require 512-bit input, we additionally hash both `rho` and `r` into an intermediate value that has a 256-bit length, and concat this value into `a_pk` to achieve 512-bit length input. Here are our own interpretation of the `k` value:
- `k = SHA256(a_pk, SHA256(rho,r))`

We calculate the `tm` value using the following code.

In [ ]:
# Get a random 256-bit value for the r
r_set = [random.getrandbits(256).to_bytes(int(256/8),"big") for x in range(2**DEPTH)]
tm_set = []
for leaf in range(2**DEPTH):
    r = r_set[leaf]
    intermediate_rho_r = hashlib.sha256(rho_set[leaf]+r_set[leaf]).digest()
    k = hashlib.sha256(administrator_public_key_list[leaf]+intermediate_rho_r).digest()
    tm = hashlib.sha256(cm_set[leaf]+k).digest()
    tm_set.append(tm)

tm_set

In [ ]:
def compile_user_knowledge_nullifier_tokenauth(private_key, public_key, zerotoken_comm_set, zerotoken_comm_index, serial_number, captoken_comm, word_comm, token_rho, token_r, token_w):
    merkle_tree = calculate_merkle_tree(zerotoken_comm_set)
    merkle_tree_root = merkle_tree[DEPTH][0]
    user = {
        'public_statement': {
            'merkle_tree_root': merkle_tree_root,
            'sn': serial_number,
            'cm': captoken_comm,
            'wm': word_comm,
            'wm_hex': word_comm.hex(),
        },
        'private_witness': {
            'private_key': private_key, #a_sk
            'public_key': public_key, #a_pk
            'rho': token_rho, # not randomize on every request
            'r': token_r, # 
            'tm': zerotoken_comm_set[zerotoken_comm_index],
            'w': token_w,
            'direction_selector': calculate_direction_selector(zerotoken_comm_index, DEPTH),
            'path': calculate_path(zerotoken_comm_index, DEPTH, merkle_tree),
        }
    }
    return user

In [ ]:
user_nullifier_tokenauth = compile_user_knowledge_nullifier_tokenauth(a_sk, a_pk, tm_set, random_user_leaf, sn, cm_set[random_user_leaf], wm, rho_set[random_user_leaf], r_set[random_user_leaf], w)
user_nullifier_tokenauth

In [ ]:
def format_nullifier_tokenauth_user_knowledge_into_zokrates_input_array(user):
    zokrates_input = [
        convert_bytes_array_to_u32_string_array(user['public_statement']['merkle_tree_root']),
        convert_bytes_array_to_u32_string_array(user['public_statement']['sn']),
        convert_bytes_array_to_u32_string_array(user['public_statement']['cm']),
        convert_bytes_array_to_u32_string_array(user['public_statement']['wm']),
        convert_bytes_array_to_u32_string_array(user['private_witness']['private_key']),
        convert_bytes_array_to_u32_string_array(user['private_witness']['public_key']),
        convert_bytes_array_to_u32_string_array(user['private_witness']['rho']),
        convert_bytes_array_to_u32_string_array(user['private_witness']['r']),
        convert_bytes_array_to_u32_string_array(user['private_witness']['tm']),
        convert_bytes_array_to_u32_string_array(user['private_witness']['w']),
        user['private_witness']['direction_selector'],
        [convert_bytes_array_to_u32_string_array(bytes_array) for bytes_array in user['private_witness']['path']],
    ]
    return zokrates_input

In [ ]:
compile_zokrates_script('nullifier_tokenauth')

In [ ]:
generate_proving_verification_key_with_trusted_setup('nullifier_tokenauth')

In [ ]:
generate_verification_contract_from_verification_key('nullifier_tokenauth')

In [ ]:
check_keys_size('nullifier_tokenauth')

In [ ]:
generate_proof_from_user_knowledge('nullifier_tokenauth',format_nullifier_tokenauth_user_knowledge_into_zokrates_input_array,'/{public_statement[wm_hex]}',user_nullifier_tokenauth)

In [ ]:
verify_generated_proof_with_verification_key('nullifier_tokenauth','/{public_statement[wm_hex]}',user_nullifier_tokenauth)

In [ ]:
nullifier_tokenauth_contract = compile_and_deploy_authentication_contract('nullifier_tokenauth', 'NullifierTokenAuth', w3)
update_current_merkle_root_of_the_auth_contract(nullifier_tokenauth_contract, user_nullifier_tokenauth['public_statement']['merkle_tree_root'], w3)
nullifier_tokenauth_proof, nullifier_tokenauth_inputs = parse_proof_output_json_into_contract_input('nullifier_tokenauth',format_nullifier_tokenauth_user_knowledge_into_zokrates_input_array,'/{public_statement[wm_hex]}',user_nullifier_tokenauth)
send_authentication_request_using_designated_agent(w3, nullifier_tokenauth_contract, nullifier_tokenauth_proof, nullifier_tokenauth_inputs, user_agent_account_address_hex)

## Delegatable NPAuth

This is a variant of NPAuth where we can assign an authentication proof to another blockchain account. It is important however to incorporate nullifier technique in this work, as we need a way to make the account is used in a limited manner. 

In [ ]:
compile_zokrates_script('delegatable_npauth')

In [ ]:
generate_proving_verification_key_with_trusted_setup('delegatable_npauth')

In [ ]:
generate_verification_contract_from_verification_key('delegatable_npauth')

In [ ]:
check_keys_size('delegatable_npauth')

In [ ]:
compile_user_knowledge_delegatable_npauth = compile_user_knowledge_npauth
format_delegatable_npauth_user_knowledge_into_zokrates_input_array = format_npauth_user_knowledge_into_zokrates_input_array
user_delegatable_npauth = user_npauth
delegatable_npauth_agent_account_address_hex = user_agent_account_address_hex

In [ ]:
generate_proof_from_user_knowledge('delegatable_npauth', format_delegatable_npauth_user_knowledge_into_zokrates_input_array, '/{public_statement[agent_account_address_hex]}', user_delegatable_npauth)

In [ ]:
verify_generated_proof_with_verification_key('delegatable_npauth','/{public_statement[agent_account_address_hex]}',user_delegatable_npauth)

In [ ]:
delegatable_npauth_contract = compile_and_deploy_authentication_contract('delegatable_npauth', 'DelegatableNPAuth', w3)
update_current_merkle_root_of_the_auth_contract(delegatable_npauth_contract, user_delegatable_npauth['public_statement']['merkle_tree_root'], w3)
delegatable_npauth_proof, delegatable_npauth_inputs = parse_proof_output_json_into_contract_input('delegatable_npauth',format_delegatable_npauth_user_knowledge_into_zokrates_input_array,'/{public_statement[agent_account_address_hex]}',user_delegatable_npauth)
send_authentication_request_using_designated_agent(w3, delegatable_npauth_contract, delegatable_npauth_proof, delegatable_npauth_inputs, delegatable_npauth_agent_account_address_hex)

# Measuring the Performance of All Authentication Schemes

In this section, we aim to compare the following performance from the previous schemes:
- Size Measurement
- Time Measurement
- Monetary Cost Measurement

For the time measurement, we aim to measure the following metrics:
- Compile Time
- Setup Time
- Proving Time
- Verification Time

For the monetary cost measurement, we aim to measure the following metrics:
- Replay Attack Prevention Execution Gas Cost
- Deployment Gas Cost
- Statement Check Gas Cost
- Verification Check Gas Cost
- Authentication Gas Cost

In [ ]:
from ttictoc import tic, toc
# We will order the scheme based on our table in the manuscript
# The following list consists of each scheme and its class name as a tuple
SCHEME_DICTIONARY = {
    'hashauth': {
        'compile_function': compile_user_knowledge_hashauth,
        'format_function': format_hashauth_user_knowledge_into_zokrates_input_array,
        'proof_subdirectory_path': '/{public_statement[random]}',
        'class_name': 'HashAuth',
        'dummy_knowledge': user_hashauth,
        'dummy_agent_hex': user_agent_account_address_hex,
        'display_name': 'HashAuth',
    },
    'nullifier_nonceauth': {
        'compile_function': compile_user_knowledge_nullifier_nonceauth,
        'format_function': format_nullifier_nonceauth_user_knowledge_into_zokrates_input_array,
        'proof_subdirectory_path': '/{public_statement[nonce_hex]}',
        'class_name': 'NullifierNonceAuth',
        'dummy_knowledge': user_nonce,
        'dummy_agent_hex': user_agent_account_address_hex,
        'display_name': 'NonceAuth',
    },
    'nullifier_randomauth': {
        'compile_function': compile_user_knowledge_nullifier_randomauth,
        'format_function': format_nullifier_randomauth_user_knowledge_into_zokrates_input_array,
        'proof_subdirectory_path': '/{public_statement[aid_hex]}',
        'class_name': 'NullifierRandomAuth',
        'dummy_knowledge': user_random,
        'dummy_agent_hex': user_agent_account_address_hex,
        'display_name': 'RandomAuth',
    },
    'npauth': {
        'compile_function': compile_user_knowledge_npauth,
        'format_function': format_npauth_user_knowledge_into_zokrates_input_array,
        'proof_subdirectory_path': '/{public_statement[agent_account_address_hex]}',
        'class_name': 'NPAuth',
        'dummy_knowledge': user_npauth,
        'dummy_agent_hex': user_agent_account_address_hex,
        'display_name': 'NPAuth (Ours)',
    },
    'nullifier_pseudoauth': {
        'compile_function': compile_user_knowledge_nullifier_pseudoauth,
        'format_function': format_nullifier_pseudoauth_user_knowledge_into_zokrates_input_array,
        'proof_subdirectory_path': '/{public_statement[nullifier_hex]}',
        'class_name': 'NullifierPseudoAuth',
        'dummy_knowledge': user_pseudo,
        'dummy_agent_hex': user_agent_account_address_hex,
        'display_name': 'PseudoAuth',
    },
    'nullifier_tokenauth': {
        'compile_function': compile_user_knowledge_nullifier_tokenauth,
        'format_function': format_nullifier_tokenauth_user_knowledge_into_zokrates_input_array,
        'proof_subdirectory_path': '/{public_statement[wm_hex]}',
        'class_name': 'NullifierTokenAuth',
        'dummy_knowledge': user_nullifier_tokenauth,
        'dummy_agent_hex': user_agent_account_address_hex,
        'display_name': 'TokenAuth',
    },
    'delegatable_npauth': {
        'compile_function': compile_user_knowledge_delegatable_npauth,
        'format_function': format_delegatable_npauth_user_knowledge_into_zokrates_input_array,
        'proof_subdirectory_path': '/{public_statement[agent_account_address_hex]}',
        'class_name': 'DelegatableNPAuth',
        'dummy_knowledge': user_delegatable_npauth,
        'dummy_agent_hex': delegatable_npauth_agent_account_address_hex,
        'display_name': 'Delegatable NPAuth (Ours)',
    },
}

## Size and Time Measurement
In this subsection, we will begin with measuring the important size in each scheme.

We aim to measure the following metrics for the size measurement:
- Circuit Size
- Proving Key Size
- Verification Key Size

For the time measurement, we aim to measure the following metrics:
- Compile Time
- Setup Time
- Proving Time
- Verification Time

In [ ]:
import pandas as pd
def record_scheme_computation_time_and_size(filename_prefix, loop_amount, inner_loop_amount):
    experiment_results = {}
    os.system('mkdir -p '+'experiment_results/scheme_time_and_size/')
    for scheme in SCHEME_DICTIONARY:
        experiment_results[scheme]= {}
        experiment_results[scheme]['loop_amount'] = loop_amount
        experiment_results[scheme]['inner_loop_amount'] = inner_loop_amount
        # Measuring Compile Time
        experiment_results[scheme]['compile_time_list'] = []
        for _ in range(loop_amount):
            tic()
            experiment_results[scheme]['circuit_size'] = compile_zokrates_script(scheme)
            experiment_results[scheme]['compile_time_list'].append(toc())
            
        # Measuring Setup Time
        experiment_results[scheme]['setup_time_list'] = []
        for _ in range(loop_amount):
            tic()
            generate_proving_verification_key_with_trusted_setup(scheme)
            experiment_results[scheme]['setup_time_list'].append(toc())

        # Measuring Key Size
        key_sizes = check_keys_size(scheme)
        experiment_results[scheme]['proving_key_size'] = key_sizes[0]
        experiment_results[scheme]['verification_key_size'] = key_sizes[1]

        # Measuring Proving Time
        experiment_results[scheme]['proving_time_list'] = []
        for _ in range(loop_amount):
            tic()
            generate_proof_from_user_knowledge(
                scheme, 
                SCHEME_DICTIONARY[scheme]['format_function'], 
                SCHEME_DICTIONARY[scheme]['proof_subdirectory_path'], 
                SCHEME_DICTIONARY[scheme]['dummy_knowledge'])
            experiment_results[scheme]['proving_time_list'].append(toc())

        # Measuring Verification Time
        experiment_results[scheme]['verification_time_list'] = []
        for _ in range(loop_amount):
            tic()
            for _ in range(inner_loop_amount):
                verify_generated_proof_with_verification_key(
                    scheme, 
                    SCHEME_DICTIONARY[scheme]['proof_subdirectory_path'], 
                    SCHEME_DICTIONARY[scheme]['dummy_knowledge'])
            experiment_results[scheme]['verification_time_list'].append(toc()/inner_loop_amount)
    with open('experiment_results/scheme_time_and_size/{0}_scheme_time_and_size.json'.format(filename_prefix), 'w') as f:
            json.dump(experiment_results, f)

In [ ]:
# record_scheme_computation_time_and_size('experiment', 10, 1000)

In [ ]:
def create_authentication_scheme_overview_table(filename_prefix):
    df = pd.read_json('experiment_results/scheme_time_and_size/{0}_scheme_time_and_size.json'.format(filename_prefix))
    df = df.transpose()
    df = df.reset_index()
    column_display_name = {
        'circuit_size': 'Circuit Size (constraints)',
        'transaction_payload': 'Proof Complexity',
        'transaction_payload_size': 'Proof Size (Bytes)',
        'proving_key_size': 'Proving Key Size (MB)',
        'verification_key_size': 'Verification Key (KB)',
    }
    column_type = {
        'circuit_size': 'int',
        'transaction_payload': 'string',
        'transaction_payload_size': 'int',
        'proving_key_size': 'float64',
        'verification_key_size': 'float64',
    }
    df['proving_key_size'] = df['proving_key_size']/1000000
    df['verification_key_size'] = df['verification_key_size']/1000
    scheme_display_name = {scheme:scheme_content['display_name'] for scheme, scheme_content in SCHEME_DICTIONARY.items()}
    scheme_name_order = ['hashauth', 'nullifier_tokenauth', 'nullifier_nonceauth', 'nullifier_pseudoauth', 'nullifier_randomauth','npauth','delegatable_npauth']
    df = df.set_index('index')
    df = df.reindex(index=scheme_name_order, columns = [key for key,value in column_display_name.items()])
    latex_string = {
        'g_1': '$\\mathbb{G}_1$',
        'g_2': '$\\mathbb{G}_1$',
        'hashed_value': '$H$',
        'c': '$c$',
    }
    transaction_payload_value = [
        '{hashed_value}+(2{g_1}+{g_2})',
        '4{hashed_value}+(2{g_1}+{g_2})',
        '2{hashed_value}+(2{g_1}+{g_2})',
        '{hashed_value}+{c}+(2{g_1}+{g_2})',
        '{hashed_value}+{c}+(2{g_1}+{g_2})',
        '{hashed_value}+{c}+(2{g_1}+{g_2})',
        '{hashed_value}+{c}+(2{g_1}+{g_2})',
    ]
    transaction_payload_size_value = [
        32+256,
        128+256,
        64+256,
        52+256,
        52+256,
        52+256,
        52+256,
    ]
    df = df.assign(
        transaction_payload=[x.format(**latex_string) for x in transaction_payload_value], 
        transaction_payload_size = transaction_payload_size_value)    
    df = df.astype(column_type)
    df = df.rename(index=scheme_display_name, columns=column_display_name)
    df = df.drop(['HashAuth','Delegatable NPAuth (Ours)'])
    df.index.name = None
    print(df.style.to_latex(hrules=True))
    return df.style.format(precision=2, thousands=",", decimal=".")

In [ ]:
create_authentication_scheme_overview_table('experiment')

In [ ]:
def create_computation_time_table(filename_prefix):
    df = pd.read_json('experiment_results/scheme_time_and_size/{0}_scheme_time_and_size.json'.format(filename_prefix))
    df = df.transpose()
    df = df.reset_index()
    df =  df.explode(['compile_time_list','setup_time_list','proving_time_list','verification_time_list'])
    agg_target = {
        "compile_time_list": ["mean", "std"],
        "setup_time_list": ["mean", "std"],
        "proving_time_list": ["mean", "std"],
        "verification_time_list": ["mean", "std"],
    }
    print(df['loop_amount'])
    df = df.groupby("index").agg(agg_target)


    df = df*1000
    df = df.map('{:,.2f}'.format)
    
    df['compile_time_mean_std'] = df['compile_time_list']['mean'] + ' $\\pm$ ' + df['compile_time_list']['std']
    df['setup_time_mean_std'] = df['setup_time_list']['mean'] + ' $\\pm$ ' + df['setup_time_list']['std']
    df['proving_time_mean_std'] = df['proving_time_list']['mean'] + ' $\\pm$ ' + df['proving_time_list']['std']
    df['verification_time_mean_std'] = df['verification_time_list']['mean'] + ' $\\pm$ ' + df['verification_time_list']['std']
    
    column_display_name = {
        'compile_time_mean_std': 'Compile Time (ms)',
        'setup_time_mean_std': 'Setup Time (ms)',
        'proving_time_mean_std': 'Proving Time (ms)',
        'verification_time_mean_std': 'Verification Time (ms)',
    }
    scheme_display_name = {scheme:scheme_content['display_name'] for scheme, scheme_content in SCHEME_DICTIONARY.items()}
    scheme_name_order = ['hashauth', 'nullifier_tokenauth', 'nullifier_nonceauth', 'nullifier_pseudoauth', 'nullifier_randomauth','npauth','delegatable_npauth']
    df = df.reindex(index=scheme_name_order)
    df = df[['compile_time_mean_std','setup_time_mean_std','proving_time_mean_std','verification_time_mean_std']]
    df = df.rename(index=scheme_display_name, columns=column_display_name)
    df = df.drop(['HashAuth','Delegatable NPAuth (Ours)'])
    df.index.name = None
    print(df.style.to_latex(hrules=True))
    return df.style

In [ ]:
create_computation_time_table('experiment')

## Gas Cost Experiment

In this experiment, we will record all the important operation gas cost for our scheme.

In [ ]:
def send_authentication_request_for_statement_check_using_designated_agent(web3_http_provider, deployed_contract, proof, inputs, designated_agent_address):
    # We invoke a special function that are used for experiment only
    tx_hash = deployed_contract.functions.statement_check_with_broadcast(proof, inputs).transact({"from": designated_agent_address})
    tx_receipt = web3_http_provider.eth.wait_for_transaction_receipt(tx_hash)
    return tx_receipt

def send_authentication_request_for_verification_check_using_designated_agent(web3_http_provider, deployed_contract, proof, inputs, designated_agent_address):
    # We invoke a special function that are used for experiment only
    tx_hash = deployed_contract.functions.verification_with_broadcast(proof, inputs).transact({"from": designated_agent_address})
    tx_receipt = web3_http_provider.eth.wait_for_transaction_receipt(tx_hash)
    return tx_receipt

def record_scheme_gas_cost(filename_prefix):
    web3_http_provider = Web3(Web3.HTTPProvider(DEVELOPMENT_HTTP_PROVIDER_URL))
    experiment_results = {}
    for scheme in SCHEME_DICTIONARY:
        experiment_results[scheme]= {}

        # Initiatiation process
        compile_zokrates_script(scheme)
        generate_proving_verification_key_with_trusted_setup(scheme)
        generate_verification_contract_from_verification_key(scheme)
        
        # Measuring Deploy Cost
        scheme_contract, deploy_tx_receipt = compile_and_deploy_authentication_contract(scheme, SCHEME_DICTIONARY[scheme]['class_name'], web3_http_provider, True)
        experiment_results[scheme]['deployment_gas_cost'] = deploy_tx_receipt['gasUsed']
        update_current_merkle_root_of_the_auth_contract(
            scheme_contract, 
            SCHEME_DICTIONARY[scheme]['dummy_knowledge']['public_statement']['merkle_tree_root'], 
            web3_http_provider)
        if scheme == 'nullifier_nonceauth':
            update_current_nonce_of_the_auth_contract(
                scheme_contract, 
                SCHEME_DICTIONARY[scheme]['dummy_knowledge']['public_statement']['nonce'], 
                web3_http_provider)
            
        # Measuring Statement Check Cost
        scheme_proof, scheme_inputs = parse_proof_output_json_into_contract_input(
            scheme, 
            SCHEME_DICTIONARY[scheme]['format_function'], 
            SCHEME_DICTIONARY[scheme]['proof_subdirectory_path'], 
            SCHEME_DICTIONARY[scheme]['dummy_knowledge'])
        experiment_results[scheme]['statement_check_gas_cost'] = send_authentication_request_for_statement_check_using_designated_agent(
            web3_http_provider,
            scheme_contract, 
            scheme_proof, 
            scheme_inputs, 
            SCHEME_DICTIONARY[scheme]['dummy_agent_hex'])['gasUsed']

        # Measuring Verfication Check Cost
        experiment_results[scheme]['verification_check_gas_cost'] = send_authentication_request_for_verification_check_using_designated_agent(
            web3_http_provider,
            scheme_contract, 
            scheme_proof, 
            scheme_inputs, 
            SCHEME_DICTIONARY[scheme]['dummy_agent_hex'])['gasUsed']

        # Measuring Authentication Check Cost
        scheme_contract = compile_and_deploy_authentication_contract(scheme, SCHEME_DICTIONARY[scheme]['class_name'], web3_http_provider)
        update_current_merkle_root_of_the_auth_contract(
            scheme_contract, 
            SCHEME_DICTIONARY[scheme]['dummy_knowledge']['public_statement']['merkle_tree_root'], 
            web3_http_provider)
        if scheme == 'nullifier_nonceauth':
            update_current_nonce_of_the_auth_contract(
                scheme_contract, 
                SCHEME_DICTIONARY[scheme]['dummy_knowledge']['public_statement']['nonce'], 
                web3_http_provider)
        experiment_results[scheme]['authentication_check_gas_cost'] = send_authentication_request_using_designated_agent(
            web3_http_provider,
            scheme_contract, 
            scheme_proof, 
            scheme_inputs, 
            SCHEME_DICTIONARY[scheme]['dummy_agent_hex'])['gasUsed']

        # Measuring Authentication Fail Check Cost
        scheme_contract = compile_and_deploy_authentication_contract(scheme, SCHEME_DICTIONARY[scheme]['class_name'], web3_http_provider)
        update_current_merkle_root_of_the_auth_contract(
            scheme_contract, 
            SCHEME_DICTIONARY[scheme]['dummy_knowledge']['public_statement']['merkle_tree_root'], 
            web3_http_provider)
        if scheme == 'nullifier_nonceauth':
            update_current_nonce_of_the_auth_contract(
                scheme_contract, 
                SCHEME_DICTIONARY[scheme]['dummy_knowledge']['public_statement']['nonce'], 
                web3_http_provider)

        _, scheme_fail_inputs = parse_proof_output_json_into_contract_input(
            scheme, 
            SCHEME_DICTIONARY[scheme]['format_function'], 
            SCHEME_DICTIONARY[scheme]['proof_subdirectory_path'], 
            SCHEME_DICTIONARY[scheme]['dummy_knowledge'])
        # We change the witness output to 0, making the witness value false, thus failing the check
        scheme_fail_inputs[-1] = 0
        experiment_results[scheme]['fail_authentication_check_gas_cost'] = send_authentication_request_using_designated_agent(
            web3_http_provider,
            scheme_contract, 
            scheme_proof, 
            scheme_fail_inputs, 
            SCHEME_DICTIONARY[scheme]['dummy_agent_hex'])['gasUsed']
    os.system('mkdir -p '+ 'experiment_results/scheme_gas_cost/')
    with open('experiment_results/scheme_gas_cost/{filename_prefix}_scheme_gas_cost.json'.format(filename_prefix=filename_prefix), 'w') as f:
            json.dump(experiment_results, f)

In [ ]:
# record_scheme_gas_cost('experiment')

In [ ]:
def create_authentication_scheme_gas_cost_table(filename_prefix):
    df = pd.read_json('experiment_results/scheme_gas_cost/{filename_prefix}_scheme_gas_cost.json'.format(filename_prefix=filename_prefix))
    df = df.transpose()
    df = df.reset_index()
    column_display_name = {
        'deployment_gas_cost': 'Deployment',
        'statement_check_gas_cost': 'Initial Check',
        'verification_check_gas_cost': 'Proof Verification',
        'authentication_check_gas_cost': 'Whole Authentication',
        'fail_authentication_check_gas_cost': 'Fail Authentication',
    }

    scheme_display_name = {scheme:scheme_content['display_name'] for scheme, scheme_content in SCHEME_DICTIONARY.items()}
    scheme_name_order = ['hashauth', 'nullifier_tokenauth', 'nullifier_nonceauth', 'nullifier_pseudoauth', 'nullifier_randomauth','npauth','delegatable_npauth']
    df = df.set_index('index')
    df = df.reindex(index=scheme_name_order, columns = [key for key,value in column_display_name.items()])
    df = df.rename(index=scheme_display_name, columns=column_display_name)
    df = df.drop(['HashAuth','Delegatable NPAuth (Ours)'])
    df.index.name = None
    return df.style.format(precision=2, thousands=",", decimal=".")

In [ ]:
print(create_authentication_scheme_gas_cost_table('experiment').to_latex(hrules=True))
create_authentication_scheme_gas_cost_table('experiment')

# Real-World Performance Experiment

We aim to measure all of our schemes against a high loaded traffic to test the latency of each schemes, which is reflected by the schemes scalability factor.

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed, ProcessPoolExecutor
from ttictoc import tic, toc
import pandas as pd
PRODUCTION_HTTP_PROVIDER_URL = 'http://127.0.0.1:8546'

In [ ]:
def send_authenticate_request(private_key, public_key, agent_address_hex, web3_http_provider, authentication_contract, scheme_name, misc_information={}):
    user_knowledge_input = {
        'private_key': private_key,
        'public_key': public_key,
    }
    user_knowledge_input.update(misc_information)
    user_knowledge = SCHEME_DICTIONARY[scheme_name]['compile_function'](**user_knowledge_input)
    generate_proof_from_user_knowledge(
        scheme_name, 
        SCHEME_DICTIONARY[scheme_name]['format_function'], 
        SCHEME_DICTIONARY[scheme_name]['proof_subdirectory_path'], 
        user_knowledge)
    user_proof, user_inputs = parse_proof_output_json_into_contract_input(
        scheme_name, 
        SCHEME_DICTIONARY[scheme_name]['format_function'], 
        SCHEME_DICTIONARY[scheme_name]['proof_subdirectory_path'], 
        user_knowledge)
    return send_authentication_request_using_designated_agent(
        web3_http_provider,
        authentication_contract, 
        user_proof,user_inputs, 
        agent_address_hex)
    

In [ ]:
def npauth_user_logic(user_index, private_key, public_key_set, user_prefunded_accounts, amount_of_devices, contract_address, contract_abi, max_tries):
    print('User {user_index} attempting to access {amount_of_devices} devices using {user_prefunded_accounts}'.format(
        user_index=user_index, 
        amount_of_devices = amount_of_devices, 
        user_prefunded_accounts=user_prefunded_accounts))
    scheme_name = 'npauth'
    user_web3_http_provider = Web3(Web3.HTTPProvider(PRODUCTION_HTTP_PROVIDER_URL))
    authentication_contract = connect_to_contract(contract_address, contract_abi, user_web3_http_provider)
    public_key = public_key_set[user_index]

    # We record the following metrics for each user
    return_value = {
        'scheme_name': scheme_name,
        'user_index': user_index,
        'amount_of_devices': amount_of_devices,
        'max_tries': max_tries,
        'successful_authentication': 0,
        'fail_authentication': 0,
        'total_gas_cost': 0,
        'total_time_elapsed': 0,
        'authentication_logs': [],
    }

    for device_index in range(amount_of_devices):
        tic()
        current_authentication_gas_cost = 0
        for _ in range(max_tries):
            agent_address_hex = user_prefunded_accounts[device_index]
            misc_information = {
                'agent_account_address_hex': agent_address_hex,
                'public_key_set': public_key_set,
                'public_key_index': user_index
            }
            tx_receipt = send_authenticate_request(
                private_key, 
                public_key, 
                agent_address_hex,
                user_web3_http_provider,
                authentication_contract,
                scheme_name,
                misc_information
            )
            return_value['total_gas_cost'] += tx_receipt['gasUsed']
            current_authentication_gas_cost += tx_receipt['gasUsed']
            if len(tx_receipt['logs'])==0:
                print('User {user_index} fail to authenticate '.format(user_index=user_index))
                return_value['fail_authentication'] += 1
            else:
                print('User {user_index} successful to authenticate '.format(user_index=user_index))
                return_value['successful_authentication'] += 1
                break
        current_time_elapsed = toc()
        return_value['authentication_logs'].append({
            'authentication_gas_cost': current_authentication_gas_cost,
            'authentication_time_elapsed': current_time_elapsed,
        })
        return_value['total_time_elapsed'] += current_time_elapsed

    
    print('User {user_index} authenticate {successful_authentication}/{amount_of_devices} devices using {scheme_name}'.format(**return_value))
    return return_value

In [ ]:
def hashauth_user_logic(user_index, private_key, public_key_set, user_prefunded_accounts, amount_of_devices, contract_address, contract_abi, max_tries):
    print('User {user_index} attempting to access {amount_of_devices} devices using {user_prefunded_accounts}'.format(
        user_index=user_index, 
        amount_of_devices = amount_of_devices, 
        user_prefunded_accounts=user_prefunded_accounts))
    
    scheme_name = 'hashauth'
    user_web3_http_provider = Web3(Web3.HTTPProvider(PRODUCTION_HTTP_PROVIDER_URL))
    authentication_contract = connect_to_contract(contract_address, contract_abi, user_web3_http_provider)
    public_key = public_key_set[user_index]

    # We record the following metrics for each user
    return_value = {
        'scheme_name': scheme_name,
        'user_index': user_index,
        'amount_of_devices': amount_of_devices,
        'max_tries': max_tries,
        'successful_authentication': 0,
        'fail_authentication': 0,
        'total_gas_cost': 0,
        'total_time_elapsed': 0,
        'authentication_logs': [],
    }

    tic()
    for device_index in range(amount_of_devices):
        tic()
        current_authentication_gas_cost = 0
        for _ in range(max_tries):
            agent_address_hex = user_prefunded_accounts[device_index]
            misc_information = {
                'agent_account_address_hex': agent_address_hex,
                'public_key_set': public_key_set,
                'public_key_index': user_index
            }
            tx_receipt = send_authenticate_request(
                private_key, 
                public_key, 
                agent_address_hex,
                user_web3_http_provider,
                authentication_contract,
                scheme_name,
                misc_information
            )
            return_value['total_gas_cost'] += tx_receipt['gasUsed']
            current_authentication_gas_cost += tx_receipt['gasUsed']
            if len(tx_receipt['logs'])==0:
                print('User {user_index} fail to authenticate '.format(user_index=user_index))
                return_value['fail_authentication'] += 1
            else:
                print('User {user_index} successful to authenticate '.format(user_index=user_index))
                return_value['successful_authentication'] += 1
                break
        current_time_elapsed = toc()
        return_value['authentication_logs'].append({
            'authentication_gas_cost': current_authentication_gas_cost,
            'authentication_time_elapsed': current_time_elapsed,
        })
        return_value['total_time_elapsed'] += current_time_elapsed
    print('User {user_index} authenticate {successful_authentication}/{amount_of_devices} devices using {scheme_name}'.format(**return_value))
    return return_value

In [ ]:
from eth_abi.exceptions import InsufficientDataBytes
from time import sleep

def nullifier_nonceauth_user_logic(user_index, private_key, public_key_set, user_prefunded_accounts, amount_of_devices, contract_address, contract_abi, max_tries):
    print('User {user_index} attempting to access {amount_of_devices} devices using {user_prefunded_accounts}'.format(
        user_index=user_index, 
        amount_of_devices = amount_of_devices, 
        user_prefunded_accounts=user_prefunded_accounts))
    
    scheme_name = 'nullifier_nonceauth'
    user_web3_http_provider = Web3(Web3.HTTPProvider(PRODUCTION_HTTP_PROVIDER_URL))
    authentication_contract = connect_to_contract(contract_address, contract_abi, user_web3_http_provider)
    public_key = public_key_set[user_index]

    # We record the following metrics for each user
    return_value = {
        'scheme_name': scheme_name,
        'user_index': user_index,
        'amount_of_devices': amount_of_devices,
        'max_tries': max_tries,
        'successful_authentication': 0,
        'fail_authentication': 0,
        'total_gas_cost': 0,
        'total_time_elapsed': 0,
        'authentication_logs': [],
    }

    tic()
    for device_index in range(amount_of_devices):
        tic()
        current_authentication_gas_cost = 0
        for _ in range(max_tries):
            agent_address_hex = user_prefunded_accounts[device_index]
            current_nonce_hex = random.getrandbits(160).to_bytes(int(160/8),"big").hex()
            try:
                current_nonce_hex = authentication_contract.functions.current_nonce().call().hex()
            except Exception as error:
                user_web3_http_provider = Web3(Web3.HTTPProvider(PRODUCTION_HTTP_PROVIDER_URL))
                print("An error occurred:", type(error).__name__, "–", error)
                
            misc_information = {
                'public_key_set': public_key_set,
                'public_key_index': user_index,
                'nonce_hex': current_nonce_hex
            }
            tx_receipt = send_authenticate_request(
                private_key, 
                public_key, 
                agent_address_hex,
                user_web3_http_provider,
                authentication_contract,
                scheme_name,
                misc_information
            )
            return_value['total_gas_cost'] += tx_receipt['gasUsed']
            current_authentication_gas_cost += tx_receipt['gasUsed']
            if len(tx_receipt['logs'])==0:
                print('User {user_index} fail to authenticate '.format(user_index=user_index))
                return_value['fail_authentication'] += 1
            else:
                print('User {user_index} successful to authenticate '.format(user_index=user_index))
                return_value['successful_authentication'] += 1
                break
        current_time_elapsed = toc()
        return_value['authentication_logs'].append({
            'authentication_gas_cost': current_authentication_gas_cost,
            'authentication_time_elapsed': current_time_elapsed,
        })
        return_value['total_time_elapsed'] += current_time_elapsed
    print('User {user_index} authenticate {successful_authentication}/{amount_of_devices} devices using {scheme_name}'.format(**return_value))
    return return_value

In [ ]:
def nullifier_randomauth_user_logic(user_index, private_key, public_key_set, user_prefunded_accounts, amount_of_devices, contract_address, contract_abi, max_tries):
    print('User {user_index} attempting to access {amount_of_devices} devices using {user_prefunded_accounts}'.format(
        user_index=user_index, 
        amount_of_devices = amount_of_devices, 
        user_prefunded_accounts=user_prefunded_accounts))
    
    scheme_name = 'nullifier_randomauth'
    user_web3_http_provider = Web3(Web3.HTTPProvider(PRODUCTION_HTTP_PROVIDER_URL))
    authentication_contract = connect_to_contract(contract_address, contract_abi, user_web3_http_provider)
    public_key = public_key_set[user_index]

    # We record the following metrics for each user
    return_value = {
        'scheme_name': scheme_name,
        'user_index': user_index,
        'amount_of_devices': amount_of_devices,
        'max_tries': max_tries,
        'successful_authentication': 0,
        'fail_authentication': 0,
        'total_gas_cost': 0,
        'total_time_elapsed': 0,
        'authentication_logs': [],
    }

    tic()
    for device_index in range(amount_of_devices):
        tic()
        current_authentication_gas_cost = 0
        for _ in range(max_tries):
            agent_address_hex = user_prefunded_accounts[device_index]
            aid_hex = random.getrandbits(160).to_bytes(20, "big").hex()
            misc_information = {
                'public_key_set': public_key_set,
                'public_key_index': user_index,
                'aid_hex': aid_hex
            }
            tx_receipt = send_authenticate_request(
                private_key, 
                public_key, 
                agent_address_hex,
                user_web3_http_provider,
                authentication_contract,
                scheme_name,
                misc_information
            )
            return_value['total_gas_cost'] += tx_receipt['gasUsed']
            current_authentication_gas_cost += tx_receipt['gasUsed']
            if len(tx_receipt['logs'])==0:
                print('User {user_index} fail to authenticate '.format(user_index=user_index))
                return_value['fail_authentication'] += 1
            else:
                print('User {user_index} successful to authenticate '.format(user_index=user_index))
                return_value['successful_authentication'] += 1
                break
        current_time_elapsed = toc()
        return_value['authentication_logs'].append({
            'authentication_gas_cost': current_authentication_gas_cost,
            'authentication_time_elapsed': current_time_elapsed,
        })
        return_value['total_time_elapsed'] += current_time_elapsed
    print('User {user_index} authenticate {successful_authentication}/{amount_of_devices} devices using {scheme_name}'.format(**return_value))
    return return_value

In [ ]:
def nullifier_pseudoauth_user_logic(user_index, private_key, public_key_set, user_prefunded_accounts, amount_of_devices, contract_address, contract_abi, max_tries):
    print('User {user_index} attempting to access {amount_of_devices} devices using {user_prefunded_accounts}'.format(
        user_index=user_index, 
        amount_of_devices = amount_of_devices, 
        user_prefunded_accounts=user_prefunded_accounts))
    
    scheme_name = 'nullifier_pseudoauth'
    user_web3_http_provider = Web3(Web3.HTTPProvider(PRODUCTION_HTTP_PROVIDER_URL))
    authentication_contract = connect_to_contract(contract_address, contract_abi, user_web3_http_provider)
    public_key = public_key_set[user_index]
    is_authenticated = False

    # We record the following metrics for each user
    return_value = {
        'scheme_name': scheme_name,
        'user_index': user_index,
        'amount_of_devices': amount_of_devices,
        'max_tries': max_tries,
        'successful_authentication': 0,
        'fail_authentication': 0,
        'total_gas_cost': 0,
        'total_time_elapsed': 0,
        'authentication_logs': [],
    }

    tic()
    for device_index in range(amount_of_devices):
        tic()
        current_authentication_gas_cost = 0
        for _ in range(max_tries):
            agent_address_hex = user_prefunded_accounts[0]
            nullifier = hashlib.sha256(private_key+int(1).to_bytes(int(256/8),"big")).digest()
            misc_information = {
                'public_key_set': public_key_set,
                'public_key_index': user_index,
                'nullifier': nullifier,
            }
            if is_authenticated:
                tx_receipt = user_web3_http_provider.eth.wait_for_transaction_receipt(
                    authentication_contract.functions.express_authentication_with_broadcast().transact({"from": agent_address_hex}))
            else:
                tx_receipt = send_authenticate_request(
                    private_key, 
                    public_key, 
                    agent_address_hex,
                    user_web3_http_provider,
                    authentication_contract,
                    scheme_name,
                    misc_information
                )
            return_value['total_gas_cost'] += tx_receipt['gasUsed']
            current_authentication_gas_cost += tx_receipt['gasUsed']
            if len(tx_receipt['logs'])==0:
                print('User {user_index} fail to authenticate '.format(user_index=user_index))
                return_value['fail_authentication'] += 1
            else:
                print('User {user_index} successful to authenticate '.format(user_index=user_index))
                is_authenticated = True
                return_value['successful_authentication'] += 1
                break
        current_time_elapsed = toc()
        return_value['authentication_logs'].append({
            'authentication_gas_cost': current_authentication_gas_cost,
            'authentication_time_elapsed': current_time_elapsed,
        })
        return_value['total_time_elapsed'] += current_time_elapsed
    print('User {user_index} authenticate {successful_authentication}/{amount_of_devices} devices using {scheme_name}'.format(**return_value))
    return return_value

In [ ]:
def nullifier_tokenauth_user_logic(
    user_index, 
    private_key, 
    public_key_set, 
    user_prefunded_accounts, 
    amount_of_devices, 
    contract_address, 
    contract_abi, 
    max_tries, 
    rho, 
    cm, 
    r, 
    tm_set):
    print('User {user_index} attempting to access {amount_of_devices} devices using {user_prefunded_accounts}'.format(
        user_index=user_index, 
        amount_of_devices = amount_of_devices, 
        user_prefunded_accounts=user_prefunded_accounts))
    
    scheme_name = 'nullifier_tokenauth'
    user_web3_http_provider = Web3(Web3.HTTPProvider(PRODUCTION_HTTP_PROVIDER_URL))
    authentication_contract = connect_to_contract(contract_address, contract_abi, user_web3_http_provider)
    public_key = public_key_set[user_index]

    # We record the following metrics for each user
    return_value = {
        'scheme_name': scheme_name,
        'user_index': user_index,
        'amount_of_devices': amount_of_devices,
        'max_tries': max_tries,
        'successful_authentication': 0,
        'fail_authentication': 0,
        'total_gas_cost': 0,
        'total_time_elapsed': 0,
        'authentication_logs': [],
    }

    tic()
    for device_index in range(amount_of_devices):
        tic()
        current_authentication_gas_cost = 0
        for _ in range(max_tries):
            agent_address_hex = user_prefunded_accounts[device_index]
            
            w = random.getrandbits(256).to_bytes(int(256/8),"big")
            k_prime = hashlib.sha256(private_key+w).digest()
            wm = hashlib.sha256(k_prime+int(0).to_bytes(32,"big")).digest()

            sn = hashlib.sha256(rho+private_key).digest()

            misc_information = {
                'zerotoken_comm_set': tm_set,
                'zerotoken_comm_index': user_index,
                'serial_number': sn,
                'captoken_comm': cm,
                'word_comm': wm,
                'token_rho': rho,
                'token_r': r,
                'token_w': w,
            }
            tx_receipt = send_authenticate_request(
                private_key, 
                public_key, 
                agent_address_hex,
                user_web3_http_provider,
                authentication_contract,
                scheme_name,
                misc_information
            )
            return_value['total_gas_cost'] += tx_receipt['gasUsed']
            current_authentication_gas_cost += tx_receipt['gasUsed']
            if len(tx_receipt['logs'])==0:
                print('User {user_index} fail to authenticate '.format(user_index=user_index))
                return_value['fail_authentication'] += 1
            else:
                print('User {user_index} successful to authenticate '.format(user_index=user_index))
                return_value['successful_authentication'] += 1
                break
        current_time_elapsed = toc()
        return_value['authentication_logs'].append({
            'authentication_gas_cost': current_authentication_gas_cost,
            'authentication_time_elapsed': current_time_elapsed,
        })
        return_value['total_time_elapsed'] += current_time_elapsed
    print('User {user_index} authenticate {successful_authentication}/{amount_of_devices} devices using {scheme_name}'.format(**return_value))
    return return_value

In [ ]:
def delegatable_npauth_user_logic(user_index, private_key, public_key_set, user_prefunded_accounts, amount_of_devices, contract_address, contract_abi, max_tries):
    print('User {user_index} attempting to access {amount_of_devices} devices using {user_prefunded_accounts}'.format(
        user_index=user_index, 
        amount_of_devices = amount_of_devices, 
        user_prefunded_accounts=user_prefunded_accounts))
    scheme_name = 'delegatable_npauth'
    user_web3_http_provider = Web3(Web3.HTTPProvider(PRODUCTION_HTTP_PROVIDER_URL))
    authentication_contract = connect_to_contract(contract_address, contract_abi, user_web3_http_provider)
    public_key = public_key_set[user_index]

    # We record the following metrics for each user
    return_value = {
        'scheme_name': scheme_name,
        'user_index': user_index,
        'amount_of_devices': amount_of_devices,
        'max_tries': max_tries,
        'successful_authentication': 0,
        'fail_authentication': 0,
        'total_gas_cost': 0,
        'total_time_elapsed': 0,
        'authentication_logs': [],
    }

    for device_index in range(amount_of_devices):
        tic()
        current_authentication_gas_cost = 0
        for _ in range(max_tries):
            agent_address_hex = user_prefunded_accounts[device_index]
            misc_information = {
                'agent_account_address_hex': agent_address_hex,
                'public_key_set': public_key_set,
                'public_key_index': user_index
            }
            tx_receipt = send_authenticate_request(
                private_key, 
                public_key, 
                agent_address_hex,
                user_web3_http_provider,
                authentication_contract,
                scheme_name,
                misc_information
            )
            return_value['total_gas_cost'] += tx_receipt['gasUsed']
            current_authentication_gas_cost += tx_receipt['gasUsed']
            if len(tx_receipt['logs'])==0:
                print('User {user_index} fail to authenticate '.format(user_index=user_index))
                return_value['fail_authentication'] += 1
            else:
                print('User {user_index} successful to authenticate '.format(user_index=user_index))
                return_value['successful_authentication'] += 1
                break
        current_time_elapsed = toc()
        return_value['authentication_logs'].append({
            'authentication_gas_cost': current_authentication_gas_cost,
            'authentication_time_elapsed': current_time_elapsed,
        })
        return_value['total_time_elapsed'] += current_time_elapsed

    
    print('User {user_index} authenticate {successful_authentication}/{amount_of_devices} devices using {scheme_name}'.format(**return_value))
    return return_value

In [ ]:
SCHEME_DICTIONARY['npauth']['user_logic_function'] = npauth_user_logic
SCHEME_DICTIONARY['hashauth']['user_logic_function'] = hashauth_user_logic
SCHEME_DICTIONARY['nullifier_nonceauth']['user_logic_function'] = nullifier_nonceauth_user_logic
SCHEME_DICTIONARY['nullifier_randomauth']['user_logic_function'] = nullifier_randomauth_user_logic
SCHEME_DICTIONARY['nullifier_pseudoauth']['user_logic_function'] = nullifier_pseudoauth_user_logic
SCHEME_DICTIONARY['nullifier_tokenauth']['user_logic_function'] = nullifier_tokenauth_user_logic
SCHEME_DICTIONARY['delegatable_npauth']['user_logic_function'] = delegatable_npauth_user_logic

In [ ]:
def experiment_logic_npauth(amount_of_users, amount_of_devices):
    scheme_name = 'npauth'
    prefunded_account_per_user = amount_of_devices * 1

    # All users generate their private key and send it to the administrator
    user_private_key_set, administrator_public_key_set = generate_public_private_key_set()
    
    # Administrator setup the system
    administrator_merkle_tree = calculate_merkle_tree(administrator_public_key_set)
    administrator_merkle_root = administrator_merkle_tree[DEPTH][0]

    compile_zokrates_script(scheme_name)
    generate_proving_verification_key_with_trusted_setup(scheme_name)
    generate_verification_contract_from_verification_key(scheme_name)
    
    administrator_web3_http_provider = Web3(Web3.HTTPProvider(PRODUCTION_HTTP_PROVIDER_URL))
    authentication_contract = compile_and_deploy_authentication_contract(
        scheme_name, 
        SCHEME_DICTIONARY[scheme_name]['class_name'], 
        administrator_web3_http_provider)
    authentication_contract_address = authentication_contract.address
    authentication_contract_abi = authentication_contract.abi
    
    update_current_merkle_root_of_the_auth_contract(
        authentication_contract, 
        administrator_merkle_root, 
        administrator_web3_http_provider)

    # Users setup before accessing the system
    prefunded_accounts = administrator_web3_http_provider.eth.accounts
    assert(len(prefunded_accounts)%prefunded_account_per_user == 0) # Make sure all users get same amount of account
    assert(len(prefunded_accounts)/prefunded_account_per_user >= amount_of_users) # Make sure the amount of prefunded accounts is enough
    user_prefunded_accounts_list = [prefunded_accounts[x:x+prefunded_account_per_user] for x in range(0, len(prefunded_accounts), prefunded_account_per_user)]
    user_max_tries = amount_of_devices # How many times the user should try to authenticate for each devices before giving up

    experiment_data = []
    
    with ProcessPoolExecutor(max_workers=amount_of_users) as thread_pool_exec:
        user_future_list = []
        for user_index in range(amount_of_users):
            user_logic_kwargs = {
                'user_index': user_index,
                'private_key': user_private_key_set[user_index],
                'public_key_set': administrator_public_key_set, # We assumed each user capable to find the public key list
                'user_prefunded_accounts': user_prefunded_accounts_list[user_index],
                'amount_of_devices': amount_of_devices,
                'contract_address': authentication_contract_address,
                'contract_abi': authentication_contract_abi,
                'max_tries': user_max_tries
            }
            user_future_list.append(thread_pool_exec.submit(SCHEME_DICTIONARY[scheme_name]['user_logic_function'], **user_logic_kwargs))
        for completed_future in as_completed(user_future_list):
            result = completed_future.result()
            experiment_data.append(result)

    return experiment_data

In [ ]:
def experiment_logic_hashauth(amount_of_users, amount_of_devices):
    scheme_name = 'hashauth'
    prefunded_account_per_user = amount_of_devices * 1

    # All users generate their private key and send it to the administrator
    user_private_key_set, administrator_public_key_set = generate_public_private_key_set()
    
    # Administrator setup the system
    administrator_merkle_tree = calculate_merkle_tree(administrator_public_key_set)
    administrator_merkle_root = administrator_merkle_tree[DEPTH][0]

    compile_zokrates_script(scheme_name)
    generate_proving_verification_key_with_trusted_setup(scheme_name)
    generate_verification_contract_from_verification_key(scheme_name)
    
    administrator_web3_http_provider = Web3(Web3.HTTPProvider(PRODUCTION_HTTP_PROVIDER_URL))
    authentication_contract = compile_and_deploy_authentication_contract(
        scheme_name, 
        SCHEME_DICTIONARY[scheme_name]['class_name'], 
        administrator_web3_http_provider)
    authentication_contract_address = authentication_contract.address
    authentication_contract_abi = authentication_contract.abi
    
    update_current_merkle_root_of_the_auth_contract(
        authentication_contract, 
        administrator_merkle_root, 
        administrator_web3_http_provider)

    # Users setup before accessing the system
    prefunded_accounts = administrator_web3_http_provider.eth.accounts
    assert(len(prefunded_accounts)%prefunded_account_per_user == 0) # Make sure all users get same amount of account
    assert(len(prefunded_accounts)/prefunded_account_per_user >= amount_of_users) # Make sure the amount of prefunded accounts is enough
    user_prefunded_accounts_list = [prefunded_accounts[x:x+prefunded_account_per_user] for x in range(0, len(prefunded_accounts), prefunded_account_per_user)]
    user_max_tries = amount_of_devices # How many times the user should try to authenticate for each devices before giving up

    experiment_data = []
    
    with ProcessPoolExecutor(max_workers=amount_of_users) as thread_pool_exec:
        user_future_list = []
        for user_index in range(amount_of_users):
            user_logic_kwargs = {
                'user_index': user_index,
                'private_key': user_private_key_set[user_index],
                'public_key_set': administrator_public_key_set, # We assumed each user capable to find the public key list
                'user_prefunded_accounts': user_prefunded_accounts_list[user_index],
                'amount_of_devices': amount_of_devices,
                'contract_address': authentication_contract_address,
                'contract_abi': authentication_contract_abi,
                'max_tries': user_max_tries
            }
            user_future_list.append(thread_pool_exec.submit(SCHEME_DICTIONARY[scheme_name]['user_logic_function'], **user_logic_kwargs))
        for completed_future in as_completed(user_future_list):
            result = completed_future.result()
            experiment_data.append(result)

    return experiment_data

In [ ]:
def experiment_logic_nullifier_nonceauth(amount_of_users, amount_of_devices):
    scheme_name = 'nullifier_nonceauth'
    prefunded_account_per_user = amount_of_devices * 1

    # All users generate their private key and send it to the administrator
    user_private_key_set, administrator_public_key_set = generate_public_private_key_set()
    
    # Administrator setup the system
    administrator_merkle_tree = calculate_merkle_tree(administrator_public_key_set)
    administrator_merkle_root = administrator_merkle_tree[DEPTH][0]

    compile_zokrates_script(scheme_name)
    generate_proving_verification_key_with_trusted_setup(scheme_name)
    generate_verification_contract_from_verification_key(scheme_name)
    
    administrator_web3_http_provider = Web3(Web3.HTTPProvider(PRODUCTION_HTTP_PROVIDER_URL))
    authentication_contract = compile_and_deploy_authentication_contract(
        scheme_name, 
        SCHEME_DICTIONARY[scheme_name]['class_name'], 
        administrator_web3_http_provider)
    authentication_contract_address = authentication_contract.address
    authentication_contract_abi = authentication_contract.abi
    
    update_current_merkle_root_of_the_auth_contract(
        authentication_contract, 
        administrator_merkle_root, 
        administrator_web3_http_provider)

    initial_nonce_bytes = random_integer.to_bytes(20, "big")
    update_current_nonce_of_the_auth_contract(
        authentication_contract, 
        initial_nonce_bytes, 
        administrator_web3_http_provider)

    # Users setup before accessing the system
    prefunded_accounts = administrator_web3_http_provider.eth.accounts
    assert(len(prefunded_accounts)%prefunded_account_per_user == 0) # Make sure all users get same amount of account
    assert(len(prefunded_accounts)/prefunded_account_per_user >= amount_of_users) # Make sure the amount of prefunded accounts is enough
    user_prefunded_accounts_list = [prefunded_accounts[x:x+prefunded_account_per_user] for x in range(0, len(prefunded_accounts), prefunded_account_per_user)]
    user_max_tries = amount_of_devices * amount_of_users # How many times the user should try to authenticate for each devices before giving up

    experiment_data = []
    
    with ProcessPoolExecutor(max_workers=amount_of_users) as thread_pool_exec:
        user_future_list = []
        for user_index in range(amount_of_users):
            user_logic_kwargs = {
                'user_index': user_index,
                'private_key': user_private_key_set[user_index],
                'public_key_set': administrator_public_key_set, # We assumed each user capable to find the public key list
                'user_prefunded_accounts': user_prefunded_accounts_list[user_index],
                'amount_of_devices': amount_of_devices,
                'contract_address': authentication_contract_address,
                'contract_abi': authentication_contract_abi,
                'max_tries': user_max_tries
            }
            user_future_list.append(thread_pool_exec.submit(SCHEME_DICTIONARY[scheme_name]['user_logic_function'], **user_logic_kwargs))
        for completed_future in as_completed(user_future_list):
            result = completed_future.result()
            experiment_data.append(result)

    return experiment_data

In [ ]:
def experiment_logic_nullifier_randomauth(amount_of_users, amount_of_devices):
    scheme_name = 'nullifier_randomauth'
    prefunded_account_per_user = amount_of_devices * 1

    # All users generate their private key and send it to the administrator
    user_private_key_set, administrator_public_key_set = generate_public_private_key_set()
    
    # Administrator setup the system
    administrator_merkle_tree = calculate_merkle_tree(administrator_public_key_set)
    administrator_merkle_root = administrator_merkle_tree[DEPTH][0]

    compile_zokrates_script(scheme_name)
    generate_proving_verification_key_with_trusted_setup(scheme_name)
    generate_verification_contract_from_verification_key(scheme_name)
    
    administrator_web3_http_provider = Web3(Web3.HTTPProvider(PRODUCTION_HTTP_PROVIDER_URL))
    authentication_contract = compile_and_deploy_authentication_contract(
        scheme_name, 
        SCHEME_DICTIONARY[scheme_name]['class_name'], 
        administrator_web3_http_provider)
    authentication_contract_address = authentication_contract.address
    authentication_contract_abi = authentication_contract.abi
    
    update_current_merkle_root_of_the_auth_contract(
        authentication_contract, 
        administrator_merkle_root, 
        administrator_web3_http_provider)

    # Users setup before accessing the system
    prefunded_accounts = administrator_web3_http_provider.eth.accounts
    assert(len(prefunded_accounts)%prefunded_account_per_user == 0) # Make sure all users get same amount of account
    assert(len(prefunded_accounts)/prefunded_account_per_user >= amount_of_users) # Make sure the amount of prefunded accounts is enough
    user_prefunded_accounts_list = [prefunded_accounts[x:x+prefunded_account_per_user] for x in range(0, len(prefunded_accounts), prefunded_account_per_user)]
    user_max_tries = amount_of_devices # How many times the user should try to authenticate for each devices before giving up

    experiment_data = []
    
    with ProcessPoolExecutor(max_workers=amount_of_users) as thread_pool_exec:
        user_future_list = []
        for user_index in range(amount_of_users):
            user_logic_kwargs = {
                'user_index': user_index,
                'private_key': user_private_key_set[user_index],
                'public_key_set': administrator_public_key_set, # We assumed each user capable to find the public key list
                'user_prefunded_accounts': user_prefunded_accounts_list[user_index],
                'amount_of_devices': amount_of_devices,
                'contract_address': authentication_contract_address,
                'contract_abi': authentication_contract_abi,
                'max_tries': user_max_tries
            }
            user_future_list.append(thread_pool_exec.submit(SCHEME_DICTIONARY[scheme_name]['user_logic_function'], **user_logic_kwargs))
        for completed_future in as_completed(user_future_list):
            result = completed_future.result()
            experiment_data.append(result)

    return experiment_data

In [ ]:
def experiment_logic_nullifier_pseudoauth(amount_of_users, amount_of_devices):
    scheme_name = 'nullifier_pseudoauth'
    prefunded_account_per_user = 1

    # All users generate their private key and send it to the administrator
    user_private_key_set, administrator_public_key_set = generate_public_private_key_set()
    
    # Administrator setup the system
    administrator_merkle_tree = calculate_merkle_tree(administrator_public_key_set)
    administrator_merkle_root = administrator_merkle_tree[DEPTH][0]

    compile_zokrates_script(scheme_name)
    generate_proving_verification_key_with_trusted_setup(scheme_name)
    generate_verification_contract_from_verification_key(scheme_name)
    
    administrator_web3_http_provider = Web3(Web3.HTTPProvider(PRODUCTION_HTTP_PROVIDER_URL))
    authentication_contract = compile_and_deploy_authentication_contract(
        scheme_name, 
        SCHEME_DICTIONARY[scheme_name]['class_name'], 
        administrator_web3_http_provider)
    authentication_contract_address = authentication_contract.address
    authentication_contract_abi = authentication_contract.abi
    
    update_current_merkle_root_of_the_auth_contract(
        authentication_contract, 
        administrator_merkle_root, 
        administrator_web3_http_provider)

    # Users setup before accessing the system
    prefunded_accounts = administrator_web3_http_provider.eth.accounts
    assert(len(prefunded_accounts)%prefunded_account_per_user == 0) # Make sure all users get same amount of account
    assert(len(prefunded_accounts)/prefunded_account_per_user >= amount_of_users) # Make sure the amount of prefunded accounts is enough
    user_prefunded_accounts_list = [prefunded_accounts[x:x+prefunded_account_per_user] for x in range(0, len(prefunded_accounts), prefunded_account_per_user)]
    user_max_tries = amount_of_devices # How many times the user should try to authenticate for each devices before giving up

    experiment_data = []
    
    with ProcessPoolExecutor(max_workers=amount_of_users) as thread_pool_exec:
        user_future_list = []
        for user_index in range(amount_of_users):
            user_logic_kwargs = {
                'user_index': user_index,
                'private_key': user_private_key_set[user_index],
                'public_key_set': administrator_public_key_set, # We assumed each user capable to find the public key list
                'user_prefunded_accounts': user_prefunded_accounts_list[user_index],
                'amount_of_devices': amount_of_devices,
                'contract_address': authentication_contract_address,
                'contract_abi': authentication_contract_abi,
                'max_tries': user_max_tries
            }
            user_future_list.append(thread_pool_exec.submit(SCHEME_DICTIONARY[scheme_name]['user_logic_function'], **user_logic_kwargs))
        for completed_future in as_completed(user_future_list):
            result = completed_future.result()
            experiment_data.append(result)

    return experiment_data

In [ ]:
def experiment_logic_nullifier_tokenauth(amount_of_users, amount_of_devices):
    scheme_name = 'nullifier_tokenauth'
    prefunded_account_per_user = amount_of_devices * 1

    # All users generate their private key and send it to the administrator
    user_private_key_set, administrator_public_key_set = generate_public_private_key_set()
    
    # Administrator setup the system
    rho_set = [random.getrandbits(256).to_bytes(int(256/8),"big") for x in range(2**DEPTH)]
    cm_set = [random.getrandbits(256).to_bytes(int(256/8),"big") for x in range(2**DEPTH)]
    r_set = [random.getrandbits(256).to_bytes(int(256/8),"big") for x in range(2**DEPTH)]
    tm_set = []
    for leaf in range(2**DEPTH):
        intermediate_rho_r = hashlib.sha256(rho_set[leaf]+r_set[leaf]).digest()
        k = hashlib.sha256(administrator_public_key_set[leaf]+intermediate_rho_r).digest()
        tm = hashlib.sha256(cm_set[leaf]+k).digest()
        tm_set.append(tm)
    
    administrator_merkle_tree = calculate_merkle_tree(tm_set)
    administrator_merkle_root = administrator_merkle_tree[DEPTH][0]

    compile_zokrates_script(scheme_name)
    generate_proving_verification_key_with_trusted_setup(scheme_name)
    generate_verification_contract_from_verification_key(scheme_name)
    
    administrator_web3_http_provider = Web3(Web3.HTTPProvider(PRODUCTION_HTTP_PROVIDER_URL))
    authentication_contract = compile_and_deploy_authentication_contract(
        scheme_name, 
        SCHEME_DICTIONARY[scheme_name]['class_name'], 
        administrator_web3_http_provider)
    authentication_contract_address = authentication_contract.address
    authentication_contract_abi = authentication_contract.abi
    
    update_current_merkle_root_of_the_auth_contract(
        authentication_contract, 
        administrator_merkle_root, 
        administrator_web3_http_provider)

    # Users setup before accessing the system
    prefunded_accounts = administrator_web3_http_provider.eth.accounts
    assert(len(prefunded_accounts)%prefunded_account_per_user == 0) # Make sure all users get same amount of account
    assert(len(prefunded_accounts)/prefunded_account_per_user >= amount_of_users) # Make sure the amount of prefunded accounts is enough
    user_prefunded_accounts_list = [prefunded_accounts[x:x+prefunded_account_per_user] for x in range(0, len(prefunded_accounts), prefunded_account_per_user)]
    user_max_tries = amount_of_devices # How many times the user should try to authenticate for each devices before giving up

    experiment_data = []
    
    with ProcessPoolExecutor(max_workers=amount_of_users) as thread_pool_exec:
        user_future_list = []
        for user_index in range(amount_of_users):
            user_logic_kwargs = {
                'user_index': user_index,
                'private_key': user_private_key_set[user_index],
                'public_key_set': administrator_public_key_set, # We assumed each user capable to find the public key list
                'user_prefunded_accounts': user_prefunded_accounts_list[user_index],
                'amount_of_devices': amount_of_devices,
                'contract_address': authentication_contract_address,
                'contract_abi': authentication_contract_abi,
                'max_tries': user_max_tries,
                'cm': cm_set[user_index],
                'rho': rho_set[user_index],
                'r': r_set[user_index],
                'tm_set': tm_set,
            }
            user_future_list.append(thread_pool_exec.submit(SCHEME_DICTIONARY[scheme_name]['user_logic_function'], **user_logic_kwargs))
        for completed_future in as_completed(user_future_list):
            result = completed_future.result()
            experiment_data.append(result)

    return experiment_data

In [ ]:
def experiment_logic_delegatable_npauth(amount_of_users, amount_of_devices):
    scheme_name = 'delegatable_npauth'
    prefunded_account_per_user = amount_of_devices * 1

    # All users generate their private key and send it to the administrator
    user_private_key_set, administrator_public_key_set = generate_public_private_key_set()
    
    # Administrator setup the system
    administrator_merkle_tree = calculate_merkle_tree(administrator_public_key_set)
    administrator_merkle_root = administrator_merkle_tree[DEPTH][0]

    compile_zokrates_script(scheme_name)
    generate_proving_verification_key_with_trusted_setup(scheme_name)
    generate_verification_contract_from_verification_key(scheme_name)
    
    administrator_web3_http_provider = Web3(Web3.HTTPProvider(PRODUCTION_HTTP_PROVIDER_URL))
    authentication_contract = compile_and_deploy_authentication_contract(
        scheme_name, 
        SCHEME_DICTIONARY[scheme_name]['class_name'], 
        administrator_web3_http_provider)
    authentication_contract_address = authentication_contract.address
    authentication_contract_abi = authentication_contract.abi
    
    update_current_merkle_root_of_the_auth_contract(
        authentication_contract, 
        administrator_merkle_root, 
        administrator_web3_http_provider)

    # Users setup before accessing the system
    prefunded_accounts = administrator_web3_http_provider.eth.accounts
    assert(len(prefunded_accounts)%prefunded_account_per_user == 0) # Make sure all users get same amount of account
    assert(len(prefunded_accounts)/prefunded_account_per_user >= amount_of_users) # Make sure the amount of prefunded accounts is enough
    user_prefunded_accounts_list = [prefunded_accounts[x:x+prefunded_account_per_user] for x in range(0, len(prefunded_accounts), prefunded_account_per_user)]
    user_max_tries = amount_of_devices # How many times the user should try to authenticate for each devices before giving up

    experiment_data = []
    
    with ProcessPoolExecutor(max_workers=amount_of_users) as thread_pool_exec:
        user_future_list = []
        for user_index in range(amount_of_users):
            user_logic_kwargs = {
                'user_index': user_index,
                'private_key': user_private_key_set[user_index],
                'public_key_set': administrator_public_key_set, # We assumed each user capable to find the public key list
                'user_prefunded_accounts': user_prefunded_accounts_list[user_index],
                'amount_of_devices': amount_of_devices,
                'contract_address': authentication_contract_address,
                'contract_abi': authentication_contract_abi,
                'max_tries': user_max_tries
            }
            user_future_list.append(thread_pool_exec.submit(SCHEME_DICTIONARY[scheme_name]['user_logic_function'], **user_logic_kwargs))
        for completed_future in as_completed(user_future_list):
            result = completed_future.result()
            experiment_data.append(result)

    return experiment_data

In [ ]:
def run_all_experiment_logic(amount_of_users_parameter, amount_of_devices_parameter, filename_prefix):
    unified_experiment = []
    unified_experiment += experiment_logic_npauth(amount_of_users_parameter, amount_of_devices_parameter)
    # Due to the vulnerability of hashauth, we remove hashauth from our work comparison
    # unified_experiment += experiment_logic_hashauth(amount_of_users_parameter, amount_of_devices_parameter)
    unified_experiment += experiment_logic_nullifier_nonceauth(amount_of_users_parameter, amount_of_devices_parameter)
    unified_experiment += experiment_logic_nullifier_randomauth(amount_of_users_parameter, amount_of_devices_parameter)
    unified_experiment += experiment_logic_nullifier_pseudoauth(amount_of_users_parameter, amount_of_devices_parameter)
    unified_experiment += experiment_logic_nullifier_tokenauth(amount_of_users_parameter, amount_of_devices_parameter)
    unified_experiment += experiment_logic_delegatable_npauth(amount_of_users_parameter, amount_of_devices_parameter)

    filename = 'experiment_results/average_gas_cost_and_latency/'+'{filename_prefix}/{filename_prefix}_{amount_of_users}_{amount_of_devices}.json'.format(filename_prefix=filename_prefix,amount_of_users=amount_of_users_parameter, amount_of_devices=amount_of_devices_parameter)
    subprocess_run_wrapper('mkdir -p '+ 'experiment_results/average_gas_cost_and_latency/'+'{filename_prefix}/'.format(filename_prefix=filename_prefix))
    with open(filename, 'w') as f:
        json.dump(unified_experiment, f)

In [ ]:
EXPERIMENT_PARAMETER_USER_AMOUNT_LIST = [1,2,4]
EXPERIMENT_PARAMETER_DEVICES_AMOUNT_LIST = [1,2,4]

# for loop in [2,3,4]:
#     for user_amount in EXPERIMENT_PARAMETER_USER_AMOUNT_LIST:
#         for device_amount in EXPERIMENT_PARAMETER_DEVICES_AMOUNT_LIST:
#             run_all_experiment_logic(user_amount,device_amount,'experiment_results_real_world_{0}'.format(loop))

In [ ]:
import seaborn
import matplotlib.pyplot as plt

In [ ]:
RENAME_COLUMN_DICTIONARY = {
    'scheme_name':'Scheme Name',
    'total_time_elapsed':'Time Elapsed (units: seconds)',
    'total_gas_cost':'Gas Cost Usage (units: Gas Cost)',
    'authentication_time_elapsed':'Authentication Time Elapsed (units: seconds)',
    'authentication_gas_cost':'Authentication Gas Cost Usage (units: Gas Cost)',
    'amount_of_users': 'Amount of Users',
    'amount_of_devices': 'Amount of Devices',
}
RENAME_SCHEME_DICTIONARY = {scheme_name:scheme['display_name'] for (scheme_name,scheme) in SCHEME_DICTIONARY.items()}

In [ ]:
def unified_visualization(filename,user_amount_list, device_amount_list, loop_list):
    df_list = []
    for loop in loop_list:
        for user_amount in user_amount_list:
            for device_amount in device_amount_list:
                df = pd.read_json('experiment_results/average_gas_cost_and_latency/{0}_{3}/{0}_{3}_{1}_{2}.json'.format(filename, user_amount, device_amount, loop))
                df['amount_of_users'] = user_amount
                df_list.append(df)
    df = pd.concat(df_list)
    df = df.explode('authentication_logs')
    df = pd.concat([df, df.authentication_logs.apply(pd.Series)], axis=1)
    df['total_gas_cost'] = df['total_gas_cost']/df['amount_of_devices']
    df['total_time_elapsed'] = df['total_time_elapsed']/df['amount_of_devices']
    df =  df[df['scheme_name'] != 'delegatable_npauth']
    df = df.replace(RENAME_SCHEME_DICTIONARY)
    df = df.rename(columns = RENAME_COLUMN_DICTIONARY)
    return df

In [ ]:
experiment_dataframe = unified_visualization('experiment_results_real_world',[1,2,4],[1,2,4], [1,2])

In [ ]:
def show_pandas_authentication_time_elapsed_results(dataframe, print_latex = False):
    df = dataframe
    agg_target = {
        RENAME_COLUMN_DICTIONARY['authentication_time_elapsed']: ['mean','std','min','max'],
    }
    df = df.groupby([RENAME_COLUMN_DICTIONARY['scheme_name'],RENAME_COLUMN_DICTIONARY['amount_of_users'],RENAME_COLUMN_DICTIONARY['amount_of_devices']]).agg(agg_target)
    df = df
    if print_latex:
        print(df.style.format(precision=2, thousands=",", decimal=".").to_latex(hrules=True))
        return
    return df.style.format(precision=2, thousands=",", decimal=".")

def show_pandas_authentication_gas_cost_results(dataframe, print_latex = False):
    df = dataframe
    agg_target = {
        RENAME_COLUMN_DICTIONARY['authentication_gas_cost']: ['mean','std','min','max'],
    }
    df = df.groupby([RENAME_COLUMN_DICTIONARY['scheme_name'],RENAME_COLUMN_DICTIONARY['amount_of_users'],RENAME_COLUMN_DICTIONARY['amount_of_devices']]).agg(agg_target)
    df = df
    if print_latex:
        print(df.style.format(precision=2, thousands=",", decimal=".").to_latex(hrules=True, multirow_align='c'))
        return
    return df.style.format(precision=2, thousands=",", decimal=".")

In [ ]:
show_pandas_authentication_time_elapsed_results(experiment_dataframe,True)

In [ ]:
show_pandas_authentication_gas_cost_results(experiment_dataframe,True)

In [ ]:
seaborn.set_theme()

In [ ]:
fig = seaborn.catplot(
    data=experiment_dataframe,
    kind='bar', 
    x=RENAME_COLUMN_DICTIONARY['amount_of_users'], 
    y=RENAME_COLUMN_DICTIONARY['authentication_time_elapsed'], 
    hue=RENAME_COLUMN_DICTIONARY['scheme_name'], 
    errorbar="ci", 
    col=RENAME_COLUMN_DICTIONARY['amount_of_devices'],
    aspect=0.8
)
fig.savefig('authentication-expected-time-elapsed.pdf')

In [ ]:
fig = seaborn.catplot(
    data=experiment_dataframe,
    kind='bar', 
    x=RENAME_COLUMN_DICTIONARY['amount_of_users'], 
    y=RENAME_COLUMN_DICTIONARY['authentication_gas_cost'], 
    hue=RENAME_COLUMN_DICTIONARY['scheme_name'], 
    errorbar="ci", 
    col=RENAME_COLUMN_DICTIONARY['amount_of_devices'],
    aspect=0.8
)
fig.savefig('authentication-expected-gas-cost.pdf')

In [ ]:
def unified_visualization_delegatable_scheme(filename,user_amount_list, device_amount_list, loop_list):
    df_list = []
    for loop in loop_list:
        for user_amount in user_amount_list:
            for device_amount in device_amount_list:
                df = pd.read_json('experiment_results/average_gas_cost_and_latency/{0}_{3}/{0}_{3}_{1}_{2}.json'.format(filename, user_amount, device_amount, loop))
                df['amount_of_users'] = user_amount
                df_list.append(df)
    df = pd.concat(df_list)
    df = df.explode('authentication_logs')
    df = pd.concat([df, df.authentication_logs.apply(pd.Series)], axis=1)
    df['total_gas_cost'] = df['total_gas_cost']/df['amount_of_devices']
    df['total_time_elapsed'] = df['total_time_elapsed']/df['amount_of_devices']
    delegatable_npauth =  df[df['scheme_name'] == 'delegatable_npauth']
    nullifier_tokenauth =  df[df['scheme_name'] == 'nullifier_tokenauth']
    df = pd.concat([delegatable_npauth, nullifier_tokenauth])
    df = df.replace(RENAME_SCHEME_DICTIONARY)
    df = df.rename(columns = RENAME_COLUMN_DICTIONARY)
    return df

In [ ]:
application_experiment = unified_visualization_delegatable_scheme('experiment_results_real_world',[1,2,4],[1,2,4], [1,2])

In [ ]:
fig = seaborn.catplot(
    data=application_experiment,
    kind='bar', 
    x=RENAME_COLUMN_DICTIONARY['amount_of_users'], 
    y=RENAME_COLUMN_DICTIONARY['authentication_time_elapsed'], 
    hue=RENAME_COLUMN_DICTIONARY['scheme_name'], 
    errorbar="ci", 
    col=RENAME_COLUMN_DICTIONARY['amount_of_devices'],
    aspect=0.7,
)
fig.savefig('application-authentication-expected-time-elapsed.pdf')

In [ ]:
fig = seaborn.catplot(
    data=application_experiment,kind='bar', 
    x=RENAME_COLUMN_DICTIONARY['amount_of_users'], 
    y=RENAME_COLUMN_DICTIONARY['authentication_gas_cost'], 
    hue=RENAME_COLUMN_DICTIONARY['scheme_name'], 
    errorbar="ci", 
    col=RENAME_COLUMN_DICTIONARY['amount_of_devices'],
    aspect=0.7
)
fig.savefig('application-authentication-expected-gas-cost.pdf')